Here is **Chapter 13: Indicator Engineering for Time-Series Systems** with comprehensive explanations and NEPSE stock prediction examples.

---

# **Chapter 13: Indicator Engineering for Time-Series Systems**

## **13.1 Understanding Domain-Specific Indicators**

Indicator engineering is the process of transforming raw time-series data into meaningful signals that capture underlying patterns, momentum, volatility, and market microstructure. In financial time-series prediction, particularly for the Nepal Stock Exchange (NEPSE), indicators serve as the bridge between raw price/volume data and machine learning models.

**Why Indicators Matter:**
- **Noise Reduction**: Raw price data contains significant noise; indicators smooth this noise while preserving signal
- **Feature Normalization**: Indicators like Z-scores and percentiles provide scale-invariant features
- **Temporal Patterns**: Moving averages and momentum indicators capture trend direction and strength
- **Predictive Power**: Properly engineered indicators often have higher correlation with future returns than raw prices

**NEPSE Data Context:**
For the NEPSE system using the CSV format: `S.No,Symbol,Conf.,Open,High,Low,Close,LTP,...`, we will engineer indicators using `Close` (closing price), `Open`, `High`, `Low`, `Vol` (volume), and `VWAP` (Volume Weighted Average Price).

---

## **13.2 Trend-Based Indicators**

Trend indicators help identify the direction and strength of price movements over time. They smooth out short-term fluctuations to reveal underlying trends.

### **13.2.1 Moving Averages**

Moving averages are the foundation of trend analysis. They calculate the average price over a specific window, smoothing out short-term volatility.

**Simple Moving Average (SMA):**
$$SMA_t = \frac{P_t + P_{t-1} + ... + P_{t-n+1}}{n}$$

Where $P_t$ is the price at time $t$ and $n$ is the window size.

```python
import pandas as pd
import numpy as np

def calculate_sma(data, window=14, price_col='Close'):
    """
    Calculate Simple Moving Average for NEPSE stock data.
    
    Parameters:
    -----------
    data : pd.DataFrame
        NEPSE data with columns including 'Close'
    window : int
        Lookback period for moving average
    price_col : str
        Column name to calculate SMA on (typically 'Close')
    
    Returns:
    --------
    pd.Series
        SMA values aligned with original index
    """
    # Calculate rolling mean using pandas
    # min_periods=1 ensures we get values even for initial rows
    sma = data[price_col].rolling(window=window, min_periods=1).mean()
    
    return sma

# Example usage with NEPSE data structure
def add_moving_averages_to_nepse(df):
    """
    Add multiple SMA indicators to NEPSE dataframe.
    Common periods: 5 (weekly), 10 (bi-weekly), 20 (monthly), 50, 200
    """
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Calculate SMAs for different time horizons
    df['SMA_5'] = calculate_sma(df, window=5)    # Short-term trend
    df['SMA_20'] = calculate_sma(df, window=20)  # Monthly trend (approx)
    df['SMA_50'] = calculate_sma(df, window=50)  # Medium-term trend
    
    # Trend direction indicator: Price relative to SMA
    df['Price_Above_SMA20'] = (df['Close'] > df['SMA_20']).astype(int)
    
    # Distance from SMA (percentage)
    df['Distance_From_SMA20'] = ((df['Close'] - df['SMA_20']) / df['SMA_20']) * 100
    
    return df

# Detailed explanation of the implementation:
# 1. rolling(window=window): Creates a rolling window object that slides over the series
# 2. mean(): Calculates the arithmetic mean for each window
# 3. min_periods=1: Ensures we calculate means even when we have fewer than 'window' observations
# 4. The result is a Series with the same index as the input, but values represent the average
#    of the current and previous (window-1) observations
```

**Code Explanation:**

The `calculate_sma` function uses pandas' `rolling` method, which creates a sliding window of size `window` over the time-series. The `mean()` method computes the average for each window position. The `min_periods=1` parameter ensures that even for the first few rows where we don't have a full window, we still calculate the average of available data.

The `add_moving_averages_to_nepse` function demonstrates practical application by calculating multiple SMAs (5, 20, 50-day) which represent different trend horizons. It also creates derived features like `Price_Above_SMA20` (a binary indicator of trend direction) and `Distance_From_SMA20` (magnitude of deviation from trend).

### **13.2.2 Exponential Moving Averages**

Exponential Moving Averages (EMA) give more weight to recent prices, making them more responsive to new information than SMAs.

**EMA Formula:**
$$EMA_t = \alpha \times P_t + (1 - \alpha) \times EMA_{t-1}$$

Where $\alpha = \frac{2}{n+1}$ and $n$ is the window period.

```python
def calculate_ema(data, window=14, price_col='Close'):
    """
    Calculate Exponential Moving Average for NEPSE data.
    
    EMA gives more weight to recent prices compared to SMA.
    Formula: EMA_t = (Price_t * k) + (EMA_yesterday * (1-k))
    where k = 2/(window+1)
    
    Parameters:
    -----------
    data : pd.DataFrame
        NEPSE stock data
    window : int
        EMA period (commonly 12 for fast, 26 for slow in MACD)
    price_col : str
        Price column to calculate EMA on
    
    Returns:
    --------
    pd.Series
        EMA values
    """
    # Calculate smoothing factor
    alpha = 2 / (window + 1)
    
    # Method 1: Using pandas ewm (exponentially weighted moving)
    ema = data[price_col].ewm(span=window, adjust=False).mean()
    # span=window specifies the decay in terms of span
    # adjust=False uses the recursive formula (traditional EMA)
    
    return ema

def calculate_macd(data, fast=12, slow=26, signal=9, price_col='Close'):
    """
    Calculate MACD (Moving Average Convergence Divergence) for NEPSE.
    
    MACD is a trend-following momentum indicator showing relationship
    between two EMAs of a security's price.
    
    Components:
    - MACD Line: EMA(fast) - EMA(slow) [typically 12 and 26]
    - Signal Line: EMA of MACD Line [typically 9]
    - Histogram: MACD Line - Signal Line
    
    Parameters:
    -----------
    data : pd.DataFrame
        NEPSE stock data
    fast : int
        Fast EMA period (default 12)
    slow : int
        Slow EMA period (default 26)
    signal : int
        Signal line EMA period (default 9)
    price_col : str
        Price column to use
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with MACD, Signal, and Histogram columns
    """
    # Calculate EMAs
    ema_fast = calculate_ema(data, window=fast, price_col=price_col)
    ema_slow = calculate_ema(data, window=slow, price_col=price_col)
    
    # Calculate MACD line
    macd_line = ema_fast - ema_slow
    
    # Calculate Signal line (EMA of MACD)
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    
    # Calculate Histogram
    histogram = macd_line - signal_line
    
    # Create result DataFrame
    result = pd.DataFrame({
        'MACD': macd_line,
        'Signal': signal_line,
        'Histogram': histogram,
        'EMA_Fast': ema_fast,
        'EMA_Slow': ema_slow
    })
    
    return result

# Detailed explanation of MACD interpretation for NEPSE:
# 1. Bullish Signal: When MACD line crosses above Signal line (Histogram turns positive)
# 2. Bearish Signal: When MACD line crosses below Signal line (Histogram turns negative)
# 3. Divergence: When price makes new high but MACD doesn't (bearish divergence)
# 4. Zero Line Crossover: MACD crossing above/below 0 indicates trend change
```

**Code Explanation:**

The `calculate_ema` function implements the Exponential Moving Average using pandas' `ewm` (exponentially weighted moving) method. The key parameter is `span=window`, which defines the decay in terms of the window size. The formula used is $\alpha = 2/(span+1)$, which matches the standard EMA calculation where recent prices receive exponentially decreasing weights.

The `calculate_macd` function demonstrates a practical application of EMAs in technical analysis. MACD (Moving Average Convergence Divergence) is calculated by subtracting the slow EMA (26-period) from the fast EMA (12-period). This difference (MACD Line) is then smoothed with a 9-period EMA to create the Signal Line. The Histogram represents the difference between MACD and Signal lines, providing a visual representation of momentum shifts. For NEPSE stocks, positive histogram values suggest bullish momentum, while negative values indicate bearish momentum.

### **13.2.3 Trend Strength Indicators**

Trend strength indicators measure how strong a trend is, helping distinguish between strong trending markets and weak, choppy markets.

**Average Directional Index (ADX):**
ADX measures trend strength on a scale of 0-100, regardless of direction. It's derived from the Directional Movement Index (DMI).

```python
def calculate_adx(data, window=14, high_col='High', low_col='Low', close_col='Close'):
    """
    Calculate Average Directional Index (ADX) for NEPSE trend strength analysis.
    
    ADX measures trend strength (0-100):
    - 0-25: Weak or no trend
    - 25-50: Strong trend
    - 50-75: Very strong trend
    - 75-100: Extremely strong trend
    
    ADX is non-directional (doesn't indicate trend direction, only strength).
    Direction is determined by +DI and -DI lines.
    
    Calculation Steps:
    1. Calculate True Range (TR)
    2. Calculate +DM and -DM (Directional Movement)
    3. Smooth TR, +DM, -DM using Wilder's smoothing
    4. Calculate +DI and -DI
    5. Calculate DX (Directional Index)
    6. Smooth DX to get ADX
    
    Parameters:
    -----------
    data : pd.DataFrame
        NEPSE OHLC data
    window : int
        Lookback period (default 14)
    high_col, low_col, close_col : str
        Column names for OHLC
    
    Returns:
    --------
    pd.DataFrame
        ADX, +DI, -DI values
    """
    # Calculate True Range (TR)
    # TR = max(High - Low, |High - PrevClose|, |Low - PrevClose|)
    high_low = data[high_col] - data[low_col]
    high_close = abs(data[high_col] - data[close_col].shift(1))
    low_close = abs(data[low_col] - data[close_col].shift(1))
    
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    
    # Calculate Directional Movement (+DM and -DM)
    # +DM = Current High - Previous High (if positive and > Current Low - Previous Low)
    # -DM = Previous Low - Current Low (if positive and > Current High - Previous High)
    
    high_diff = data[high_col].diff()
    low_diff = data[low_col].diff()
    
    plus_dm = ((high_diff > 0) & (high_diff > low_diff)) * high_diff
    minus_dm = ((low_diff > 0) & (low_diff > high_diff)) * low_diff
    
    # Wilder's smoothing (RMA - Running Moving Average)
    # First value is simple average, subsequent values use smoothing factor
    def wilder_smoothing(series, period):
        """Apply Wilder's smoothing (exponential smoothing with alpha = 1/period)"""
        return series.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    
    # Smooth TR, +DM, -DM
    atr = wilder_smoothing(tr, window)
    plus_di = 100 * wilder_smoothing(plus_dm, window) / atr
    minus_di = 100 * wilder_smoothing(minus_dm, window) / atr
    
    # Calculate DX (Directional Index)
    # DX = 100 * |+DI - -DI| / (+DI + -DI)
    dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)
    
    # Calculate ADX (smoothed DX)
    adx = wilder_smoothing(dx, window)
    
    return pd.DataFrame({
        'ADX': adx,
        'Plus_DI': plus_di,
        'Minus_DI': minus_di,
        'ATR': atr  # Average True Range (bonus)
    })

# Example usage with NEPSE data:
def analyze_nepse_trend_strength(df):
    """
    Apply ADX analysis to NEPSE stock data to determine trend strength.
    
    Interpretation for NEPSE trading:
    - ADX < 20: Weak trend - avoid trend-following strategies, use mean reversion
    - ADX 20-40: Trend developing - good for breakout strategies
    - ADX > 40: Strong trend - ride the trend but watch for exhaustion
    
    Directional indicators:
    - Plus_DI > Minus_DI: Bullish trend
    - Plus_DI < Minus_DI: Bearish trend
    """
    # Calculate ADX with 14-day period (standard)
    adx_data = calculate_adx(df, window=14)
    
    # Combine with original data
    analysis_df = pd.concat([df, adx_data], axis=1)
    
    # Generate trading signals based on trend strength
    analysis_df['Trend_Strength'] = pd.cut(
        analysis_df['ADX'], 
        bins=[0, 20, 40, 60, 100],
        labels=['Weak', 'Developing', 'Strong', 'Extreme']
    )
    
    # Trend direction signal
    analysis_df['Trend_Direction'] = np.where(
        analysis_df['Plus_DI'] > analysis_df['Minus_DI'],
        'Bullish',
        'Bearish'
    )
    
    return analysis_df

# Detailed explanation of ADX components for NEPSE:
# 1. True Range (TR): Measures volatility by considering gaps between sessions
# 2. +DI: Measures upward trend strength (based on High prices)
# 3. -DI: Measures downward trend strength (based on Low prices)
# 4. ADX: Average of DX (directional index), smoothed over time
```

**Detailed Explanation:**

The `calculate_adx` function implements the complete Average Directional Index calculation pipeline specifically tailored for NEPSE stock data. 

**Step 1: True Range Calculation**
The True Range (TR) captures the greatest of three measurements: the current high-low range, the absolute distance from today's high to yesterday's close, or today's low to yesterday's close. This accounts for gap-up or gap-down openings common in NEPSE stocks.

**Step 2: Directional Movement**
+DM (Positive Directional Movement) measures upward pressure by comparing today's high to yesterday's high, but only if this difference exceeds the downward movement (yesterday's low minus today's low). Conversely, -DM measures downward pressure. This helps identify whether NEPSE stocks are gaining bullish or bearish momentum.

**Step 3: Wilder's Smoothing**
Unlike simple moving averages, Wilder's smoothing applies an exponential moving average with a smoothing factor of $\frac{1}{n}$. This gives more weight to recent data while maintaining stability, crucial for NEPSE's sometimes volatile but trending nature.

**Step 4: Directional Indices**
+DI and -DI normalize the directional movements by the Average True Range, converting them to percentages (0-100). When +DI > -DI for a NEPSE stock, it indicates bullish dominance; when reversed, bearish pressure dominates.

**Step 5: ADX Calculation**
The Directional Index (DX) measures the absolute difference between +DI and -DI relative to their sum, giving the strength of trend regardless of direction. Smoothing DX with Wilder's method yields ADX. For NEPSE trading:
- **ADX < 20**: The stock is ranging (sideways), avoid trend-following
- **ADX 20-40**: Trend developing, suitable for breakout entries
- **ADX > 40**: Strong trend, ride it but watch for exhaustion signals

### **13.2.2 Exponential Moving Averages**

While covered in the ADX section above, EMAs deserve specific attention for NEPSE trend analysis. The EMA reacts faster to price changes than SMA, making it ideal for capturing early trend reversals in Nepali stocks.

**Key Differences for NEPSE:**
- **EMA 12 vs EMA 26**: The difference forms the MACD line, widely used in NEPSE technical analysis
- **Signal Line**: 9-day EMA of MACD, used to generate buy/sell signals when MACD crosses above/below

### **13.2.3 Trend Strength Indicators**

Beyond ADX, trend strength can be measured using:

**Linear Regression Slope:**
Measures the angle and strength of the trend line fitted to recent prices.

```python
def calculate_trend_strength_linear(data, window=20, price_col='Close'):
    """
    Calculate trend strength using linear regression slope.
    
    For NEPSE stocks, the slope indicates:
    - Positive slope: Uptrend (buying pressure)
    - Negative slope: Downtrend (selling pressure)
    - Near zero: Sideways/ranging
    
    Also calculates R-squared to measure how well the trend explains price movement.
    """
    from scipy import stats
    
    prices = data[price_col]
    
    def linear_regression_slope(x):
        """Calculate slope for a window of prices"""
        if len(x) < 2:
            return 0
        x_vals = np.arange(len(x))
        slope, intercept, r_value, p_value, std_err = stats.linregress(x_vals, x)
        return slope
    
    # Calculate rolling slope
    slope = prices.rolling(window=window).apply(
        lambda x: linear_regression_slope(x), 
        raw=True
    )
    
    # Calculate R-squared (trend reliability)
    r_squared = prices.rolling(window=window).apply(
        lambda x: stats.linregress(np.arange(len(x)), x).rvalue ** 2 if len(x) > 1 else 0,
        raw=True
    )
    
    # Normalize slope by price level to make comparable across stocks
    normalized_slope = (slope / prices) * 100  # Percentage change per period
    
    return pd.DataFrame({
        'Trend_Slope': slope,
        'Trend_R2': r_squared,
        'Trend_Strength_Pct': normalized_slope,
        'Trend_Direction': np.where(slope > 0, 'Up', 
                                   np.where(slope < 0, 'Down', 'Flat'))
    })
```

**Explanation:**

The `calculate_trend_strength_linear` function performs linear regression on rolling windows of NEPSE closing prices. 

**Slope Calculation:**
For each window of 20 days (default), it fits a line $y = mx + b$ where $x$ is time (0, 1, 2, ..., 19) and $y$ is the price. The slope $m$ indicates how many rupees the stock gains per day on average. A slope of +2.5 means the stock is appreciating by Rs. 2.5 per day over the 20-day period.

**R-Squared (Trend Reliability):**
The $R^2$ value (0 to 1) measures how well the linear model explains price variance. For NEPSE stocks:
- $R^2 > 0.7$: Strong linear trend (reliable for trading)
- $R^2 = 0.3-0.7$: Moderate trend (use with caution)
- $R^2 < 0.3$: No clear trend (avoid trend-following)

**Normalized Slope:**
Dividing slope by current price converts absolute rupee changes to percentage changes, making trend strength comparable across different NEPSE stocks (e.g., comparing a Rs. 500 stock vs a Rs. 50 stock).

---

## **13.3 Momentum-Based Indicators**

Momentum indicators measure the speed of price changes, helping identify overbought or oversold conditions and potential reversal points.

### **13.3.1 Rate of Change (ROC)**

ROC measures the percentage change in price over a specified period, indicating momentum direction and speed.

```python
def calculate_roc(data, window=12, price_col='Close'):
    """
    Calculate Rate of Change (ROC) for NEPSE stocks.
    
    ROC = ((Current Price - Price n periods ago) / Price n periods ago) * 100
    
    Interpretation for NEPSE:
    - ROC > 0: Bullish momentum (price higher than n periods ago)
    - ROC < 0: Bearish momentum (price lower than n periods ago)
    - ROC crossing above 0: Buy signal (momentum shift)
    - ROC crossing below 0: Sell signal
    
    Parameters:
    -----------
    data : pd.DataFrame
        NEPSE stock data
    window : int
        Lookback period (typically 12 for monthly, 25 for yearly in trading days)
    price_col : str
        Price column to calculate ROC on
    
    Returns:
    --------
    pd.Series
        ROC values as percentages
    """
    # Calculate ROC using pandas pct_change
    # pct_change calculates (current - previous)/previous
    roc = data[price_col].pct_change(periods=window) * 100
    
    # Alternative manual calculation:
    # current = data[price_col]
    # past = data[price_col].shift(window)
    # roc = ((current - past) / past) * 100
    
    return roc

def calculate_roc_signals(data, roc_col='ROC_12'):
    """
    Generate trading signals based on ROC momentum.
    
    Signals:
    - Buy: ROC crosses above 0 (momentum turning positive)
    - Sell: ROC crosses below 0 (momentum turning negative)
    - Overbought: ROC > 10 (strong upward momentum, potential reversal)
    - Oversold: ROC < -10 (strong downward momentum, potential bounce)
    """
    signals = pd.DataFrame(index=data.index)
    
    # Current ROC
    signals['ROC'] = data[roc_col]
    
    # Signal generation
    signals['Signal'] = 0
    signals.loc[signals['ROC'] > 0, 'Signal'] = 1   # Bullish
    signals.loc[signals['ROC'] < 0, 'Signal'] = -1  # Bearish
    
    # Change in signal (crossover detection)
    signals['Signal_Change'] = signals['Signal'].diff()
    
    # Buy/Sell triggers
    signals['Buy'] = signals['Signal_Change'] == 2   # -1 to 1
    signals['Sell'] = signals['Signal_Change'] == -2  # 1 to -1
    
    # Momentum extremes
    signals['Overbought'] = signals['ROC'] > 10
    signals['Oversold'] = signals['ROC'] < -10
    
    return signals
```

**Explanation:**

The `calculate_roc` function computes the Rate of Change, a pure momentum oscillator. For NEPSE stocks, ROC tells us how fast prices are changing relative to a past point (typically 12 days for short-term, 25 days for medium-term).

**Mathematical Logic:**
The formula $\frac{P_t - P_{t-n}}{P_{t-n}} \times 100$ converts absolute price changes into percentage terms. If a NEPSE stock was Rs. 100 twelve days ago and is now Rs. 110, ROC = 10%, indicating 10% upward momentum.

**Trading Interpretation:**
- **Zero Line Crossovers**: When ROC crosses above zero, it indicates the stock is performing better than it was n-periods ago (bullish). Crossing below zero indicates deterioration (bearish).
- **Extreme Values**: ROC > +10 suggests overbought conditions (price advanced too fast, likely to correct). ROC < -10 suggests oversold (potential bounce).
- **Divergences**: If NEPSE price makes a new high but ROC makes a lower high, it indicates weakening momentum (bearish divergence).

The `calculate_roc_signals` function automates signal generation by detecting crossovers (when ROC changes sign) and identifying extreme momentum conditions that might precede reversals in Nepali stocks.

### **13.3.2 Relative Strength Index (RSI)**

RSI is a momentum oscillator that measures the speed and change of price movements, oscillating between 0 and 100.

```python
def calculate_rsi(data, window=14, price_col='Close'):
    """
    Calculate Relative Strength Index (RSI) for NEPSE stocks.
    
    RSI measures momentum on a scale of 0-100:
    - RSI > 70: Overbought (potential sell signal)
    - RSI < 30: Oversold (potential buy signal)
    - RSI 50: Neutral
    
    Formula:
    RSI = 100 - (100 / (1 + RS))
    RS = Average Gain / Average Loss
    
    Parameters:
    -----------
    data : pd.DataFrame
        NEPSE stock data
    window : int
        Lookback period (default 14 days)
    price_col : str
        Price column to calculate RSI on
    
    Returns:
    --------
    pd.Series
        RSI values (0-100)
    """
    # Calculate price changes
    delta = data[price_col].diff()
    
    # Separate gains and losses
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    # Calculate average gain and loss using Wilder's smoothing
    avg_gain = gain.ewm(com=window-1, min_periods=window, adjust=False).mean()
    avg_loss = loss.ewm(com=window-1, min_periods=window, adjust=False).mean()
    
    # Calculate RS (Relative Strength)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Handle edge case where avg_loss is 0 (perfect uptrend)
    rsi = rsi.fillna(100)
    
    return rsi

def calculate_rsi_divergence(data, rsi_col='RSI_14', price_col='Close', window=5):
    """
    Detect RSI divergences for NEPSE stocks.
    
    Bullish Divergence: Price makes lower low, RSI makes higher low
    Bearish Divergence: Price makes higher high, RSI makes lower high
    
    Divergences often precede trend reversals.
    """
    # Find local extrema
    data['Price_High'] = data[price_col].rolling(window=window, center=True).max() == data[price_col]
    data['Price_Low'] = data[price_col].rolling(window=window, center=True).min() == data[price_col]
    
    data['RSI_High'] = data[rsi_col].rolling(window=window, center=True).max() == data[rsi_col]
    data['RSI_Low'] = data[rsi_col].rolling(window=window, center=True).min() == data[rsi_col]
    
    # Detect divergences
    data['Bullish_Divergence'] = (
        data['Price_Low'] & 
        (data[rsi_col] > data[rsi_col].shift(1))
    )
    
    data['Bearish_Divergence'] = (
        data['Price_High'] & 
        (data[rsi_col] < data[rsi_col].shift(1))
    )
    
    return data[['Bullish_Divergence', 'Bearish_Divergence']]
```

**Explanation:**

The `calculate_rsi` function implements the Relative Strength Index, a momentum oscillator crucial for NEPSE technical analysis. Unlike simple price-based indicators, RSI normalizes momentum to a 0-100 scale, making it comparable across different stocks regardless of absolute price levels (whether a Rs. 500 stock or Rs. 50 stock).

**Mathematical Logic:**
The algorithm first decomposes price changes into gains (positive changes) and losses (absolute values of negative changes). It then applies Wilder's smoothing (an exponential moving average with $\alpha = \frac{1}{window}$) to both series. The Relative Strength (RS) is the ratio of average gains to average losses. Finally, RSI normalizes this ratio to the 0-100 scale using $100 - \frac{100}{1+RS}$.

**NEPSE-Specific Interpretation:**
- **Overbought (>70)**: NEPSE stocks often exhibit strong momentum during bull markets. RSI > 70 suggests the stock is overextended and due for a correction or consolidation.
- **Oversold (<30)**: During market panics or corrections, RSI < 30 indicates capitulation selling, often presenting buying opportunities for fundamentally strong Nepali companies.
- **Divergences**: When NEPSE price makes a new high but RSI forms a lower high (bearish divergence), it signals weakening buying pressure despite higher prices, often preceding corrections. Conversely, bullish divergences (lower price lows, higher RSI lows) indicate accumulation.

The `calculate_rsi_divergence` function automates the detection of these divergence patterns by identifying local price extrema (peaks and troughs) using rolling window comparisons, then checking if RSI confirms or contradicts these price movements.

---

## **13.3.3 Momentum Acceleration**

Momentum acceleration measures how quickly momentum itself is changing, indicating increasing or decreasing trend velocity.

```python
def calculate_momentum_acceleration(data, roc_window=12, smooth_window=3, price_col='Close'):
    """
    Calculate momentum acceleration for NEPSE stocks.
    
    Momentum Acceleration = Rate of Change of ROC
    or Second derivative of price
    
    Interpretation:
    - Positive acceleration: Momentum increasing (trend strengthening)
    - Negative acceleration: Momentum decreasing (trend weakening)
    - Zero acceleration: Constant momentum (steady trend)
    
    For NEPSE: Helps identify when a trend is gaining steam or losing power.
    """
    # Calculate first momentum (ROC)
    roc = data[price_col].pct_change(periods=roc_window) * 100
    
    # Calculate acceleration (change in ROC)
    # Smooth it to reduce noise
    acceleration = roc.diff(periods=smooth_window)
    
    # Alternative: Second derivative of price (curvature)
    first_diff = data[price_col].diff()
    second_diff = first_diff.diff()
    
    # Normalize by price level for cross-stock comparison
    normalized_accel = (second_diff / data[price_col]) * 100
    
    return pd.DataFrame({
        'ROC': roc,
        'Momentum_Acceleration': acceleration,
        'Price_Curvature': second_diff,
        'Normalized_Acceleration': normalized_accel,
        'Acceleration_Signal': np.where(acceleration > 0, 'Increasing',
                                       np.where(acceleration < 0, 'Decreasing', 'Stable'))
    })

def detect_momentum_shifts(data, accel_col='Momentum_Acceleration', threshold=2.0):
    """
    Detect significant momentum shifts for NEPSE trading signals.
    
    When acceleration changes sign or exceeds threshold, it indicates
    potential trend continuation or reversal.
    """
    accel = data[accel_col]
    
    # Detect zero crossings (momentum shift)
    zero_crossing = np.sign(accel) != np.sign(accel.shift(1))
    
    # Detect extreme acceleration (unsustainable momentum)
    extreme_positive = accel > accel.rolling(50).mean() + threshold * accel.rolling(50).std()
    extreme_negative = accel < accel.rolling(50).mean() - threshold * accel.rolling(50).std()
    
    return pd.DataFrame({
        'Momentum_Shift': zero_crossing,
        'Extreme_Momentum': extreme_positive | extreme_negative,
        'Shift_Type': np.where(zero_crossing & (accel > 0), 'Bullish_Shift',
                              np.where(zero_crossing & (accel < 0), 'Bearish_Shift', 'None'))
    })
```

**Explanation:**

The `calculate_momentum_acceleration` function computes the second derivative of price movement, essentially measuring how fast the momentum itself is changing. In physics terms, if price is position and ROC (Rate of Change) is velocity, then acceleration is the change in velocity.

**Mathematical Foundation:**
The function calculates ROC (first momentum) as the percentage change over `roc_window` periods. Then it computes the difference (derivative) of ROC over `smooth_window` periods to get acceleration. Alternatively, it calculates the second difference of raw prices (curvature).

**NEPSE Application:**
For Nepali stocks, momentum acceleration helps identify:
1. **Trend Exhaustion**: When a strong uptrend shows negative acceleration (decelerating momentum), even though prices are still rising, the trend is losing power and may reverse soon.
2. **Trend Emergence**: Positive acceleration during a downtrend indicates selling pressure is intensifying (accelerating decline).
3. **Sustainable Moves**: Steady acceleration (constant positive or negative) suggests institutional participation in NEPSE stocks, indicating sustainable trends.

The `detect_momentum_shifts` function identifies critical turning points where acceleration changes sign (zero crossings), indicating shifts from accelerating to decelerating momentum or vice versa. These shifts often precede price reversals in NEPSE stocks by 1-3 days.

---

## **13.4 Volatility and Range Indicators**

Volatility indicators measure the degree of variation in trading prices over time. For NEPSE, volatility indicators help assess risk and identify potential breakout or consolidation periods.

### **13.4.1 Standard Deviation**

Standard deviation measures dispersion of prices around the mean, quantifying volatility.

```python
def calculate_volatility_indicators(data, window=20, price_col='Close'):
    """
    Calculate various volatility indicators for NEPSE risk assessment.
    
    Standard Deviation measures how much prices deviate from the average.
    Higher values indicate higher volatility/risk.
    
    For NEPSE:
    - Low volatility (< 2%): Stable stock, low risk, potential consolidation
    - Medium volatility (2-5%): Normal trading range
    - High volatility (> 5%): High risk/reward, potential breakout/breakdown
    """
    # Calculate returns
    returns = data[price_col].pct_change()
    
    # Standard Deviation of returns (volatility)
    std_dev = returns.rolling(window=window).std()
    
    # Annualized volatility (assuming 252 trading days per year for NEPSE)
    annualized_vol = std_dev * np.sqrt(252) * 100  # Convert to percentage
    
    # Standard Deviation of prices (absolute volatility)
    price_std = data[price_col].rolling(window=window).std()
    
    # Coefficient of Variation (relative volatility)
    # CV = StdDev / Mean, useful for comparing volatility across different priced stocks
    cv = price_std / data[price_col].rolling(window=window).mean()
    
    # Volatility regime classification
    vol_mean = annualized_vol.rolling(window=window*2).mean()
    vol_std = annualized_vol.rolling(window=window*2).std()
    
    regime = pd.cut(annualized_vol, 
                    bins=[0, vol_mean - vol_std, vol_mean + vol_std, float('inf')],
                    labels=['Low_Vol', 'Normal_Vol', 'High_Vol'])
    
    return pd.DataFrame({
        'Returns': returns,
        'Volatility': std_dev,
        'Annualized_Vol_Pct': annualized_vol,
        'Price_StdDev': price_std,
        'Coeff_of_Variation': cv,
        'Volatility_Regime': regime
    })

def calculate_bollinger_bands(data, window=20, num_std=2, price_col='Close'):
    """
    Calculate Bollinger Bands for NEPSE trend and volatility analysis.
    
    Bollinger Bands consist of:
    - Middle Band: SMA (Simple Moving Average)
    - Upper Band: SMA + (Standard Deviation * num_std)
    - Lower Band: SMA - (Standard Deviation * num_std)
    
    Interpretation for NEPSE:
    - Price touching Upper Band: Potentially overbought (consider selling)
    - Price touching Lower Band: Potentially oversold (consider buying)
    - Squeeze (bands narrowing): Low volatility, often precedes breakout
    - Expansion (bands widening): High volatility, trend continuation likely
    """
    # Calculate middle band (SMA)
    middle_band = data[price_col].rolling(window=window).mean()
    
    # Calculate standard deviation
    std_dev = data[price_col].rolling(window=window).std()
    
    # Calculate upper and lower bands
    upper_band = middle_band + (std_dev * num_std)
    lower_band = middle_band - (std_dev * num_std)
    
    # Calculate Bandwidth (% distance between bands relative to middle)
    bandwidth = ((upper_band - lower_band) / middle_band) * 100
    
    # Calculate %B (position within bands)
    # %B = 0 at lower band, 1 at upper band, 0.5 at middle
    pct_b = (data[price_col] - lower_band) / (upper_band - lower_band)
    
    # Generate signals
    signals = pd.DataFrame(index=data.index)
    signals['BB_Signal'] = 'Hold'
    signals.loc[pct_b > 0.8, 'BB_Signal'] = 'Overbought'  # Near upper band
    signals.loc[pct_b < 0.2, 'BB_Signal'] = 'Oversold'     # Near lower band
    
    # Squeeze detection (bandwidth at 6-month low)
    squeeze_threshold = bandwidth.rolling(window=120).min()
    signals['Squeeze'] = bandwidth <= squeeze_threshold * 1.05  # Within 5% of lowest
    
    return pd.DataFrame({
        'Close': data[price_col],
        'BB_Middle': middle_band,
        'BB_Upper': upper_band,
        'BB_Lower': lower_band,
        'BB_Width': bandwidth,
        'BB_PercentB': pct_b,
        'BB_Signal': signals['BB_Signal'],
        'BB_Squeeze': signals['Squeeze']
    })

# Usage example for NEPSE:
# df = pd.read_csv('nepse_stock.csv')
# df = calculate_bollinger_bands(df, window=20, num_std=2)
```

**Explanation:**

The `calculate_bollinger_bands` function implements one of the most versatile technical indicators for NEPSE analysis. 

**Mathematical Construction:**
The middle band is the 20-day SMA (Simple Moving Average), serving as the baseline trend. The upper and lower bands are positioned 2 standard deviations away from the middle band. Statistically, this captures approximately 95% of price action assuming normal distribution, meaning prices outside these bands are statistically extreme.

**NEPSE-Specific Interpretation:**
- **PercentB (%B)**: This metric normalizes price position within the bands to a 0-1 scale. For NEPSE stocks, when %B > 0.8, the stock is in the upper 20% of its recent volatility range, suggesting overbought conditions. Conversely, %B < 0.2 suggests oversold conditions suitable for accumulation.
- **Bandwidth**: Calculated as $\frac{Upper - Lower}{Middle} \times 100$, this measures volatility. A bandwidth squeeze (narrowing bands) in NEPSE often precedes significant breakouts due to low liquidity periods followed by news-driven movements.
- **Squeeze Detection**: The function identifies when bandwidth is at a 120-day (6-month) low, indicating impending volatility expansion. For Nepali markets, this often correlates with quarterly earnings announcements or regulatory news.

### **13.3.2 Relative Strength Index (RSI)**

RSI measures the magnitude of recent price changes to evaluate overbought or oversold conditions.

```python
def calculate_rsi(data, window=14, price_col='Close'):
    """
    Calculate Relative Strength Index (RSI) for NEPSE momentum analysis.
    
    RSI Formula:
    RSI = 100 - (100 / (1 + RS))
    RS = Average Gain / Average Loss
    
    Standard Interpretation:
    - RSI > 70: Overbought (consider selling)
    - RSI < 30: Oversold (consider buying)
    - RSI 50: Neutral line
    
    NEPSE Considerations:
    - In strong bull markets, RSI may stay >70 for extended periods
    - In bear markets, RSI may stay <30 for extended periods
    - Divergences between RSI and price are strong signals
    """
    # Calculate price differences
    delta = data[price_col].diff()
    
    # Separate gains and losses
    gain = delta.copy()
    loss = delta.copy()
    
    gain[gain < 0] = 0      # Keep only positive changes
    loss[loss > 0] = 0      # Keep only negative changes
    loss = abs(loss)        # Convert to positive values
    
    # Calculate average gain and loss using Wilder's smoothing
    # Wilder's method uses exponential moving average with alpha = 1/window
    avg_gain = gain.ewm(com=window-1, adjust=False, min_periods=window).mean()
    avg_loss = loss.ewm(com=window-1, adjust=False, min_periods=window).mean()
    
    # Calculate RS (Relative Strength)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Handle case where avg_loss is 0 (perfect uptrend)
    rsi = rsi.fillna(100)
    
    return rsi

def calculate_rsi_features(data, window=14):
    """
    Calculate advanced RSI-based features for NEPSE prediction models.
    
    Features include:
    - Raw RSI value
    - RSI trend (slope)
    - RSI position relative to 30/70 levels
    - Divergence signals
    """
    df = data.copy()
    
    # Basic RSI
    df['RSI'] = calculate_rsi(df, window=window)
    
    # RSI smoothed (to reduce noise)
    df['RSI_Smooth'] = df['RSI'].rolling(window=3).mean()
    
    # RSI slope (momentum of momentum)
    df['RSI_Slope'] = df['RSI'].diff(3) / 3
    
    # Position features
    df['RSI_Zone'] = pd.cut(df['RSI'], 
                            bins=[0, 30, 50, 70, 100],
                            labels=['Oversold', 'Bullish', 'Bearish', 'Overbought'])
    
    # Distance from extremes
    df['RSI_Distance_30'] = df['RSI'] - 30  # Positive = above oversold
    df['RSI_Distance_70'] = df['RSI'] - 70  # Negative = below overbought
    
    # Stochastic RSI (RSI normalized to 0-100 scale)
    rsi_min = df['RSI'].rolling(window=window).min()
    rsi_max = df['RSI'].rolling(window=window).max()
    df['Stoch_RSI'] = (df['RSI'] - rsi_min) / (rsi_max - rsi_min + 1e-10) * 100
    
    return df
```

**Explanation:**

The `calculate_rsi` function implements the Relative Strength Index using Wilder's smoothing method, which is the standard approach in technical analysis.

**Calculation Process:**
1. **Price Differencing**: Computes day-to-day price changes using `diff()`.
2. **Gain/Loss Separation**: Separates positive changes (gains) from negative changes (losses). Losses are converted to absolute values.
3. **Wilder's Smoothing**: Unlike simple moving averages, Wilder's method applies exponential smoothing with $\alpha = \frac{1}{window}$. This gives more weight to recent observations while maintaining the influence of older data, preventing sudden jumps when old data drops out of the window.
4. **Relative Strength**: RS is the ratio of average gains to average losses. If average losses are zero (perfect uptrend), RS approaches infinity.
5. **Normalization**: The formula $100 - \frac{100}{1+RS}$ normalizes RS to a 0-100 scale. When RS is high (strong gains), RSI approaches 100. When RS is low (strong losses), RSI approaches 0.

**NEPSE-Specific Features:**
The `calculate_rsi_features` function creates derived features particularly useful for Nepali stock prediction:
- **RSI_Slope**: Measures whether momentum is accelerating or decelerating. Positive slope indicates strengthening momentum.
- **RSI_Zone**: Categorizes RSI into actionable regions (Oversold <30, Bullish 30-50, Bearish 50-70, Overbought >70).
- **Stochastic RSI**: Normalizes RSI itself to a 0-100 scale over the lookback period, making it more sensitive to short-term reversals in NEPSE's sometimes range-bound market.

---

## **13.4 Volatility and Range Indicators**

Volatility indicators measure the magnitude of price fluctuations, essential for risk management and position sizing in NEPSE trading.

### **13.4.1 Standard Deviation**

Standard deviation quantifies dispersion of returns around the mean, serving as the foundation for many volatility indicators.

```python
def calculate_volatility_metrics(data, window=20, price_col='Close'):
    """
    Calculate comprehensive volatility metrics for NEPSE risk management.
    
    Includes:
    - Rolling Standard Deviation (absolute and percentage)
    - Annualized Volatility
    - Parkinson Volatility (using High-Low range)
    - Garman-Klass Volatility (using OHLC)
    
    Parameters:
    -----------
    data : pd.DataFrame
        NEPSE data with OHLC columns
    window : int
        Lookback period for volatility calculation
    price_col : str
        Column to calculate volatility on
    
    Returns:
    --------
    pd.DataFrame
        Various volatility measures
    """
    # Calculate returns
    returns = data[price_col].pct_change()
    
    # Standard deviation of returns (historical volatility)
    std_returns = returns.rolling(window=window).std()
    
    # Annualized volatility (assuming 252 trading days for NEPSE)
    ann_vol = std_returns * np.sqrt(252) * 100  # As percentage
    
    # Standard deviation of prices (absolute volatility)
    std_prices = data[price_col].rolling(window=window).std()
    
    # Coefficient of Variation (relative volatility)
    cv = std_prices / data[price_col].rolling(window=window).mean()
    
    # Parkinson Volatility (uses High-Low range, more efficient)
    # Formula: sqrt(1/(4*ln(2)) * sum(log(High/Low)^2) / n)
    log_hl = np.log(data[high_col] / data[low_col])
    parkinson_vol = np.sqrt(
        (log_hl**2).rolling(window=window).mean() / (4 * np.log(2))
    ) * np.sqrt(252) * 100  # Annualized
    
    # Garman-Klass Volatility (uses OHLC, most efficient)
    # Formula: sqrt(0.5*log(High/Low)^2 - (2*log(2)-1)*log(Close/Open)^2)
    log_ho = np.log(data[high_col] / data[open_col])
    log_lo = np.log(data[low_col] / data[open_col])
    log_co = np.log(data[close_col] / data[open_col])
    
    gk_vol = np.sqrt(
        (0.5 * (log_ho - log_lo)**2 - (2*np.log(2) - 1) * log_co**2).rolling(window=window).mean()
    ) * np.sqrt(252) * 100
    
    # Volatility regime classification
    vol_mean = ann_vol.rolling(window=window*2).mean()
    vol_std = ann_vol.rolling(window=window*2).std()
    
    regime = pd.cut(ann_vol,
                   bins=[0, vol_mean - vol_std, vol_mean + vol_std, float('inf')],
                   labels=['Low_Vol', 'Normal_Vol', 'High_Vol'])
    
    return pd.DataFrame({
        'Returns_Std': std_returns,
        'Annualized_Vol_Pct': ann_vol,
        'Price_StdDev': std_prices,
        'Coeff_of_Variation': cv,
        'Parkinson_Vol': parkinson_vol,
        'Garman_Klass_Vol': gk_vol,
        'Volatility_Regime': regime,
        'Vol_Z_Score': (ann_vol - vol_mean) / vol_std
    })

def calculate_volatility_signals(data, vol_col='Annualized_Vol_Pct', price_col='Close'):
    """
    Generate trading signals based on volatility patterns for NEPSE.
    
    Strategies:
    1. Volatility Breakout: Low vol followed by expansion = trend start
    2. Volatility Contraction: High vol followed by contraction = trend end
    3. Volatility Mean Reversion: Extreme vol readings revert to mean
    """
    vol = data[vol_col]
    
    # Detect volatility squeeze (Bollinger Bands on volatility)
    vol_bb = calculate_bollinger_bands(data.assign(Close=vol), window=20, num_std=2)
    
    # Volatility breakout signals
    squeeze = vol < vol_bb['BB_Lower']  # Volatility below lower band = squeeze
    expansion = vol > vol_bb['BB_Upper']  # Volatility above upper band = expansion
    
    # Volatility regime transitions
    vol_ma = vol.rolling(window=20).mean()
    vol_increasing = vol > vol_ma
    vol_decreasing = vol < vol_ma
    
    return pd.DataFrame({
        'Vol_Squeeze': squeeze,
        'Vol_Expansion': expansion,
        'Vol_Increasing': vol_increasing,
        'Vol_Decreasing': vol_decreasing,
        'Vol_BB_Upper': vol_bb['BB_Upper'],
        'Vol_BB_Lower': vol_bb['BB_Lower']
    })
```

**Explanation:**

The `calculate_volatility_metrics` function provides a comprehensive volatility analysis suite essential for risk management in NEPSE trading. 

**Standard Deviation vs. Parkinson vs. Garman-Klass:**
The function calculates three types of volatility estimators:
1. **Close-to-Close (Standard)**: Uses only closing prices. Simple but ignores intraday information (High/Low/Open).
2. **Parkinson**: Uses High-Low range. More efficient (uses intraday extremes) but ignores opening gaps.
3. **Garman-Klass**: Uses full OHLC data. Most statistically efficient estimator, accounting for opening gaps and intraday range.

For NEPSE stocks, Garman-Klass provides the most accurate volatility estimate because it captures the true range of price movement during the trading session (9:30 AM - 3:00 PM NPT).

**Volatility Regime Classification:**
The function classifies volatility into Low, Normal, and High regimes using statistical thresholds (mean ± standard deviation). This is crucial for NEPSE because:
- **Low Volatility**: Often precedes major moves (earnings announcements, regulatory changes)
- **High Volatility**: Usually occurs during market crashes or bubbles; indicates high risk but potential high reward
- **Normal Volatility**: Suitable for standard trend-following strategies

The `calculate_volatility_signals` function applies Bollinger Bands to the volatility series itself (volatility of volatility), identifying "volatility squeezes"—periods of unusually low volatility that historically precede significant price breakouts in NEPSE stocks.

### **13.4.2 Bollinger Bands**

Bollinger Bands were covered in the volatility section above, but they serve dual purposes as both volatility and trend indicators. The key insight for NEPSE is the "Bollinger Band Squeeze"—when the bands narrow to a 6-month low, it indicates a volatility contraction that typically precedes a significant directional move (breakout or breakdown).

### **13.4.3 Average True Range (ATR)**

ATR measures market volatility by decomposing the entire range of an asset price for that period.

```python
def calculate_atr(data, window=14, high_col='High', low_col='Low', close_col='Close'):
    """
    Calculate Average True Range (ATR) for NEPSE risk management.
    
    True Range is the greatest of:
    1. Current High - Current Low
    2. |Current High - Previous Close|
    3. |Current Low - Previous Close|
    
    ATR is the average of True Range over the window period.
    
    NEPSE Applications:
    - Position sizing: Riskier stocks (high ATR) get smaller positions
    - Stop loss placement: 2-3x ATR below entry for long positions
    - Volatility breakout: Entry when price moves > 1.5x ATR from open
    """
    # Calculate True Range components
    high_low = data[high_col] - data[low_col]
    high_close = abs(data[high_col] - data[close_col].shift(1))
    low_close = abs(data[low_col] - data[close_col].shift(1))
    
    # True Range is the maximum of the three
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    
    # Average True Range (Wilder's smoothing method)
    atr = true_range.ewm(alpha=1/window, min_periods=window, adjust=False).mean()
    
    # Alternative: Simple moving average of TR
    atr_sma = true_range.rolling(window=window).mean()
    
    # ATR-based indicators
    # 1. ATR Percentage (relative to price)
    atr_pct = (atr / data[close_col]) * 100
    
    # 2. ATR Bands (similar to Bollinger but using ATR)
    atr_upper = data[close_col] + (2 * atr)
    atr_lower = data[close_col] - (2 * atr)
    
    # 3. Normalized ATR (Chandelier Exit component)
    # Measures how current ATR compares to historical ATR
    atr_mean = atr.rolling(window=window*2).mean()
    atr_std = atr.rolling(window=window*2).std()
    atr_zscore = (atr - atr_mean) / atr_std
    
    return pd.DataFrame({
        'True_Range': true_range,
        'ATR': atr,
        'ATR_SMA': atr_sma,
        'ATR_Pct': atr_pct,
        'ATR_Upper': atr_upper,
        'ATR_Lower': atr_lower,
        'ATR_ZScore': atr_zscore,
        'ATR_Regime': pd.cut(atr_zscore, 
                            bins=[-np.inf, -1, 1, np.inf],
                            labels=['Low_Vol', 'Normal', 'High_Vol'])
    })

def calculate_chandelier_exit(data, atr_window=22, multiplier=3, 
                             high_col='High', low_col='Low', close_col='Close'):
    """
    Calculate Chandelier Exit for NEPSE trend following.
    
    Chandelier Exit sets a trailing stop based on ATR.
    - Long Exit: Highest High since entry - (ATR * multiplier)
    - Short Exit: Lowest Low since entry + (ATR * multiplier)
    
    For NEPSE long-only trading:
    Use as dynamic stop-loss that adjusts to volatility.
    """
    # Calculate ATR
    atr_data = calculate_atr(data, window=atr_window, 
                            high_col=high_col, low_col=low_col, close_col=close_col)
    atr = atr_data['ATR']
    
    # Calculate highest high and lowest low over the ATR window
    highest_high = data[high_col].rolling(window=atr_window).max()
    lowest_low = data[low_col].rolling(window=atr_window).min()
    
    # Chandelier Exit levels
    chandelier_long = highest_high - (atr * multiplier)
    chandelier_short = lowest_low + (atr * multiplier)
    
    # Distance to exit (risk measure)
    distance_to_exit_long = data[close_col] - chandelier_long
    
    return pd.DataFrame({
        'Chandelier_Long': chandelier_long,
        'Chandelier_Short': chandelier_short,
        'Highest_High_22': highest_high,
        'Lowest_Low_22': lowest_low,
        'Distance_to_Exit': distance_to_exit_long,
        'ATR_Used': atr
    })
```

**Explanation:**

The `calculate_atr` function computes the Average True Range, a critical risk management tool for NEPSE trading. 

**True Range Logic:**
Unlike simple range (High - Low), True Range accounts for gaps between trading sessions. For NEPSE, which operates 5 days a week (Sunday-Thursday), gaps can occur due to overnight news. True Range captures the greatest of:
1. Today's high-low range (normal intraday volatility)
2. Distance from today's high to yesterday's close (gap up scenario)
3. Distance from yesterday's close to today's low (gap down scenario)

**Wilder's Smoothing:**
The function applies exponential smoothing with $\alpha = \frac{1}{14}$ (for default window), which creates a smoother volatility measure than simple moving averages. This prevents whipsaws during NEPSE's sometimes erratic short-term movements.

**Chandelier Exit Application:**
The `calculate_chandelier_exit` function uses ATR for dynamic stop-loss placement. For NEPSE long positions, the exit level is set at the highest high over 22 days (approx 1 month of trading) minus 3x ATR. This trailing stop adjusts to volatility—tightening during calm periods to protect profits and widening during volatile periods to avoid whipsaws.

### **13.3.3 Momentum Acceleration**

Covered in section 13.3.1 with the `calculate_momentum_acceleration` function, this measures the second derivative of price—how quickly momentum itself is changing. For NEPSE, acceleration signals often precede major moves by 1-2 days, making them valuable for entry timing.

---

## **13.4 Volatility and Range Indicators**

### **13.4.4 Range Percentages**

Range percentages normalize daily trading ranges relative to opening prices or previous closes, enabling comparison across different priced NEPSE stocks.

```python
def calculate_range_indicators(data, open_col='Open', high_col='High', 
                              low_col='Low', close_col='Close', prev_close_col='Prev. Close'):
    """
    Calculate range-based indicators for NEPSE intraday analysis.
    
    Range indicators measure daily volatility relative to price levels.
    Useful for identifying high-volatility days and potential reversals.
    """
    # True Range (max of three measurements)
    tr1 = data[high_col] - data[low_col]
    tr2 = abs(data[high_col] - data[prev_close_col])
    tr3 = abs(data[low_col] - data[prev_close_col])
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # Range as percentage of previous close
    range_pct = (true_range / data[prev_close_col]) * 100
    
    # Intraday range percentage (High-Low)/Open
    intraday_range_pct = ((data[high_col] - data[low_col]) / data[open_col]) * 100
    
    # Position in range (where did price close within the daily range?)
    # 0 = closed at low, 100 = closed at high
    position_in_range = ((data[close_col] - data[low_col]) / 
                        (data[high_col] - data[low_col])) * 100
    
    # Handle case where high == low (no range)
    position_in_range = position_in_range.fillna(50)  # Assume middle if no range
    
    # Gap analysis (overnight gaps for NEPSE)
    gap_pct = ((data[open_col] - data[prev_close_col]) / data[prev_close_col]) * 100
    gap_type = pd.cut(gap_pct, 
                     bins=[-np.inf, -2, -0.5, 0.5, 2, np.inf],
                     labels=['Large_Down_Gap', 'Small_Down_Gap', 'No_Gap', 
                            'Small_Up_Gap', 'Large_Up_Gap'])
    
    return pd.DataFrame({
        'True_Range': true_range,
        'Range_Pct': range_pct,
        'Intraday_Range_Pct': intraday_range_pct,
        'Position_in_Range': position_in_range,
        'Gap_Pct': gap_pct,
        'Gap_Type': gap_type,
        'Is_Large_Range': range_pct > range_pct.rolling(window=20).mean() + 
                         2 * range_pct.rolling(window=20).std()
    })
```

**Explanation:**

The `calculate_range_indicators` function provides granular analysis of daily price ranges for NEPSE stocks, crucial for understanding intraday volatility patterns.

**True Range Calculation:**
Unlike simple High-Low range, True Range accounts for gaps between sessions. For NEPSE, which trades Sunday-Thursday, overnight news can cause significant gaps. True Range captures the maximum of:
1. Today's high minus today's low (normal day)
2. Absolute distance from today's high to yesterday's close (gap up and pullback)
3. Absolute distance from yesterday's close to today's low (gap down and bounce)

**Position in Range:**
This metric normalizes the closing price within the daily range to a 0-100 scale. Values near 100 indicate strong buying pressure (closed at daily high), typical of bullish sentiment in NEPSE. Values near 0 indicate distribution (closed at low). Values around 50 suggest indecision.

**Gap Analysis:**
For NEPSE, gaps >2% often indicate significant news (quarterly results, regulatory changes, sector news). The function categorizes gaps into five types, from large down gaps (panic selling) to large up gaps (euphoria), helping identify emotional extremes in the Nepali market.

---

## **13.5 Volume/Intensity Indicators**

Volume indicators confirm price movements and measure the intensity of trading activity, crucial for validating trends in NEPSE's relatively thin market.

### **13.5.1 Volume Ratios**

Volume ratios compare current volume to historical averages to identify unusual activity.

```python
def calculate_volume_indicators(data, vol_col='Vol', close_col='Close', 
                               open_col='Open', high_col='High', low_col='Low'):
    """
    Calculate volume-based indicators for NEPSE liquidity analysis.
    
    Volume indicators help confirm price trends and identify accumulation/distribution.
    Critical for NEPSE due to lower liquidity compared to major exchanges.
    """
    df = data.copy()
    
    # Basic volume moving averages
    df['Vol_SMA_5'] = df[vol_col].rolling(window=5).mean()
    df['Vol_SMA_20'] = df[vol_col].rolling(window=20).mean()
    df['Vol_SMA_50'] = df[vol_col].rolling(window=50).mean()
    
    # Volume Ratio (current vs average)
    df['Volume_Ratio'] = df[vol_col] / df['Vol_SMA_20']
    
    # Volume intensity classification
    df['Volume_Intensity'] = pd.cut(df['Volume_Ratio'],
                                   bins=[0, 0.5, 1.0, 2.0, float('inf')],
                                   labels=['Very_Low', 'Below_Avg', 'Above_Avg', 'Very_High'])
    
    # On-Balance Volume (OBV) - cumulative volume flow
    df['Price_Change'] = df[close_col].diff()
    df['Volume_Flow'] = np.where(df['Price_Change'] > 0, df[vol_col],
                                  np.where(df['Price_Change'] < 0, -df[vol_col], 0))
    df['OBV'] = df['Volume_Flow'].cumsum()
    
    # OBV moving average for signals
    df['OBV_SMA'] = df['OBV'].rolling(window=20).mean()
    df['OBV_Signal'] = np.where(df['OBV'] > df['OBV_SMA'], 'Accumulation', 'Distribution')
    
    # Volume-Weighted Average Price (VWAP) deviation
    # VWAP is provided in NEPSE data, calculate deviation from it
    if 'VWAP' in df.columns:
        df['VWAP_Deviation'] = ((df[close_col] - df['VWAP']) / df['VWAP']) * 100
        df['Above_VWAP'] = df[close_col] > df['VWAP']  # Bullish if above VWAP
    
    # Money Flow Index (MFI) - volume-weighted RSI
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    raw_money_flow = typical_price * df[vol_col]
    
    money_flow_positive = np.where(typical_price > typical_price.shift(1), raw_money_flow, 0)
    money_flow_negative = np.where(typical_price < typical_price.shift(1), raw_money_flow, 0)
    
    money_flow_ratio = pd.Series(money_flow_positive).rolling(window=14).sum() / \
                     pd.Series(money_flow_negative).rolling(window=14).sum()
    
    df['MFI'] = 100 - (100 / (1 + money_flow_ratio))
    
    # Klinger Volume Oscillator (KVO)
    # Combines volume with trend direction
    dm = ((df[high_col] + df[low_col] + df[close_col]) / 3) - \
         ((df[high_col].shift(1) + df[low_col].shift(1) + df[close_col].shift(1)) / 3)
    cm = df[vol_col] * np.where(dm > 0, 1, -1) * abs(dm)
    
    df['KVO'] = cm.ewm(span=34, adjust=False).mean() - cm.ewm(span=55, adjust=False).mean()
    
    return df
```

**Explanation:**

The `calculate_volume_indicators` function provides a comprehensive volume analysis toolkit essential for NEPSE due to the exchange's lower liquidity compared to major international markets.

**Volume Ratio Analysis:**
The `Volume_Ratio` compares current trading volume to the 20-day average. In NEPSE, volume spikes (ratio > 2.0) often indicate:
- **Institutional Activity**: Large block trades by mutual funds or insurance companies
- **News Impact**: Earnings releases, regulatory announcements, or sector news
- **Breakout Confirmation**: Price breaking resistance on high volume validates the breakout

**On-Balance Volume (OBV):**
OBV is a cumulative indicator that adds volume on up days and subtracts volume on down days. For NEPSE stocks:
- **Rising OBV**: Accumulation phase (smart money buying)
- **Falling OBV**: Distribution phase (smart money selling)
- **OBV Divergence**: When price makes new high but OBV doesn't, it indicates weak buying pressure and potential reversal

**Money Flow Index (MFI):**
MFI combines price and volume to measure buying/selling pressure. Unlike RSI which only looks at price, MFI weights the RSI calculation by volume. MFI > 80 indicates strong buying pressure (overbought), while MFI < 20 indicates heavy selling (oversold).

**Klinger Volume Oscillator (KVO):**
KVO uses volume force (volume × trend × absolute daily range) and calculates the difference between short-term (34-period) and long-term (55-period) EMAs of volume force. Positive KVO indicates accumulation, negative indicates distribution. Crossovers signal trend changes.

### **13.5.2 Volume-Weighted Metrics**

Volume-weighted metrics give more importance to periods with higher trading activity.

```python
def calculate_volume_weighted_metrics(data, vol_col='Vol', close_col='Close', 
                                     high_col='High', low_col='Low', open_col='Open'):
    """
    Calculate volume-weighted price metrics for NEPSE.
    
    Volume-weighted metrics are more reliable than simple averages because
    they account for where most trading activity occurred.
    """
    # Volume Weighted Average Price (VWAP) - cumulative
    # VWAP = cumulative(TP * Volume) / cumulative(Volume)
    # where TP = (High + Low + Close) / 3
    typical_price = (data[high_col] + data[low_col] + data[close_col]) / 3
    tp_vol = typical_price * data[vol_col]
    
    cum_tp_vol = tp_vol.cumsum()
    cum_vol = data[vol_col].cumsum()
    
    vwap_cumulative = cum_tp_vol / cum_vol
    
    # Reset VWAP periodically (e.g., monthly for NEPSE)
    # In practice, institutional traders reset VWAP at month start
    data['Month'] = pd.to_datetime(data.index).to_period('M')
    monthly_vwap = data.groupby('Month').apply(
        lambda x: (x[close_col] * x[vol_col]).cumsum() / x[vol_col].cumsum()
    ).reset_index(level=0, drop=True)
    
    # Volume Weighted Moving Average (VWMA)
    # Similar to SMA but weighted by volume
    vwma = (data[close_col] * data[vol_col]).rolling(window=20).sum() / \
           data[vol_col].rolling(window=20).sum()
    
    # Volume Weighted Standard Deviation
    vw_mean = vwma
    vw_variance = ((data[close_col] - vw_mean) ** 2 * data[vol_col]).rolling(window=20).sum() / \
                  data[vol_col].rolling(window=20).sum()
    vw_std = np.sqrt(vw_variance)
    
    # Price relative to VWAP (intraday trend strength)
    vwap_deviation = ((data[close_col] - vwap_cumulative) / vwap_cumulative) * 100
    
    # Volume Profile (where did most volume trade?)
    # Simplified: Compare current volume to VWAP distance
    volume_profile_signal = np.where(
        (data[vol_col] > data[vol_col].rolling(20).mean()) & 
        (abs(vwap_deviation) < 0.5),
        'High_Vol_At_VWAP',  # Accumulation/distribution at fair value
        'Normal'
    )
    
    return pd.DataFrame({
        'VWAP_Cumulative': vwap_cumulative,
        'VWAP_Monthly': monthly_vwap,
        'VWMA_20': vwma,
        'VW_StdDev': vw_std,
        'VWAP_Deviation_Pct': vwap_deviation,
        'Volume_Profile_Signal': volume_profile_signal,
        'Above_VWAP': data[close_col] > vwap_cumulative
    })
```

**Explanation:**

The `calculate_volume_weighted_metrics` function computes Volume Weighted Average Price (VWAP) and related metrics, which are crucial for NEPSE because they identify where the majority of trading volume occurred, indicating "fair value" according to market participants.

**VWAP Calculation:**
VWAP is calculated as the cumulative sum of (Typical Price × Volume) divided by cumulative volume. Typical Price = (High + Low + Close)/3 represents the "center of gravity" for the day. Unlike simple moving averages, VWAP gives more weight to periods with heavy institutional trading.

**NEPSE-Specific Insights:**
- **Institutional Benchmark**: In NEPSE, mutual funds and insurance companies often use VWAP as a benchmark for execution quality. Prices above VWAP suggest aggressive buying (paying premium), while prices below suggest aggressive selling.
- **Support/Resistance**: VWAP acts as dynamic support/resistance. In uptrends, price tends to bounce off VWAP; in downtrends, rallies often fail at VWAP.
- **Monthly Reset**: The function calculates monthly VWAP because NEPSE institutional investors often rebalance monthly. The monthly VWAP serves as a fair value anchor for the month.

**Volume Profile Analysis:**
The function identifies "High Volume at VWAP" days—when volume exceeds the 20-day average but price stays within 0.5% of VWAP. These days indicate accumulation or distribution by large players at fair value, often preceding significant moves.

---

## **13.5 Volume/Intensity Indicators**

### **13.5.3 On-Balance Volume (OBV)**

OBV is a cumulative indicator that uses volume flow to predict changes in stock price.

```python
def calculate_obv_advanced(data, vol_col='Vol', close_col='Close'):
    """
    Calculate On-Balance Volume (OBV) and derived signals for NEPSE.
    
    OBV Formula:
    - If Close > Previous Close: OBV = Previous OBV + Current Volume
    - If Close < Previous Close: OBV = Previous OBV - Current Volume
    - If Close = Previous Close: OBV = Previous OBV
    
    Interpretation:
    - Rising OBV: Accumulation (volume on up days > volume on down days)
    - Falling OBV: Distribution (volume on down days > volume on up days)
    - Divergence: Price rising but OBV falling = weak trend (distribution)
    
    NEPSE Specific:
    - OBV breaks above 20-day MA: Institutional accumulation
    - OBV divergence lasting >10 days: High probability reversal
    """
    # Calculate price direction
    price_change = data[close_col].diff()
    
    # Calculate OBV
    obv = pd.Series(index=data.index, dtype=float)
    obv.iloc[0] = data[vol_col].iloc[0]  # Initialize with first volume
    
    for i in range(1, len(data)):
        if price_change.iloc[i] > 0:
            obv.iloc[i] = obv.iloc[i-1] + data[vol_col].iloc[i]
        elif price_change.iloc[i] < 0:
            obv.iloc[i] = obv.iloc[i-1] - data[vol_col].iloc[i]
        else:
            obv.iloc[i] = obv.iloc[i-1]
    
    # OBV Moving Average
    obv_sma = obv.rolling(window=20).mean()
    
    # OBV Slope (momentum of volume flow)
    obv_slope = obv.diff(5)  # 5-period change
    
    # OBV Divergence detection
    price_slope = data[close_col].diff(5)
    
    # Bullish divergence: Price down, OBV up (accumulation during decline)
    bullish_div = (price_slope < 0) & (obv_slope > 0)
    
    # Bearish divergence: Price up, OBV down (distribution during rally)
    bearish_div = (price_slope > 0) & (obv_slope < 0)
    
    # OBV Trend confirmation
    obv_above_ma = obv > obv_sma
    
    return pd.DataFrame({
        'OBV': obv,
        'OBV_SMA20': obv_sma,
        'OBV_Slope': obv_slope,
        'OBV_Above_MA': obv_above_ma,
        'OBV_Bullish_Div': bullish_div,
        'OBV_Bearish_Div': bearish_div,
        'OBV_Signal': np.where(bullish_div, 'Accumulation',
                              np.where(bearish_div, 'Distribution', 'Neutral'))
    })
```

**Explanation:**

The `calculate_obv_advanced` function implements On-Balance Volume, a cumulative flow-of-funds indicator essential for NEPSE due to the market's susceptibility to institutional manipulation and low float scenarios.

**OBV Mechanics:**
The algorithm maintains a running total where volume is added on up-days and subtracted on down-days. The logic is simple but powerful: if price closes higher than yesterday, all of today's volume is considered "positive volume" (buying pressure). If price closes lower, all volume is "negative volume" (selling pressure).

**NEPSE Institutional Flow Detection:**
In the Nepali market, OBV is particularly valuable because:
1. **Smart Money Tracking**: When NEPSE price is declining but OBV is rising (bullish divergence), it indicates accumulation by informed investors (promoters, institutions) who know the stock is undervalued. This often precedes positive news announcements.
2. **Distribution Warning**: When price makes new highs but OBV fails to confirm (bearish divergence), it suggests institutional distribution—large players selling to retail investors at peaks. This is a high-probability sell signal in NEPSE.
3. **Trend Confirmation**: When both price and OBV are making higher highs and higher lows, the uptrend is healthy and sustained by volume.

**Advanced Features:**
The function calculates OBV slope (5-period rate of change) to measure the intensity of accumulation/distribution. It also detects divergences by comparing price slope to OBV slope—when they move in opposite directions for 5 consecutive periods, it flags potential reversal points.

### **13.5.2 Volume-Weighted Metrics**

Volume-weighted metrics provide a more accurate picture of where the majority of trading occurred and at what prices.

```python
def calculate_volume_profile(data, close_col='Close', vol_col='Vol', 
                            high_col='High', low_col='Low', n_bins=10):
    """
    Calculate Volume Profile for NEPSE stocks.
    
    Volume Profile shows how much volume was traded at each price level.
    Helps identify:
    - Point of Control (POC): Price level with highest volume (fair value)
    - Value Area: Price range containing 70% of volume
    - Support/Resistance levels based on volume clusters
    
    NEPSE Application:
    - POC acts as magnet for price (gravitational pull)
    - Breakout above value area high with volume = strong bullish
    - Breakdown below value area low with volume = strong bearish
    """
    # Create price bins (levels)
    price_min = data[low_col].min()
    price_max = data[high_col].max()
    bins = np.linspace(price_min, price_max, n_bins)
    
    # Assign each trade to a price level (using Close as proxy)
    data['Price_Level'] = pd.cut(data[close_col], bins=bins, labels=False)
    
    # Calculate volume at each level
    volume_profile = data.groupby('Price_Level')[vol_col].sum()
    
    # Point of Control (price level with max volume)
    poc_level = volume_profile.idxmax()
    poc_price = (bins[poc_level] + bins[poc_level + 1]) / 2
    
    # Calculate Value Area (70% of volume)
    total_volume = volume_profile.sum()
    target_volume = total_volume * 0.70
    
    # Sort levels by volume and accumulate until 70%
    sorted_levels = volume_profile.sort_values(ascending=False)
    cumsum = sorted_levels.cumsum()
    value_area_levels = sorted_levels[cumsum <= target_volume].index
    
    value_area_low = bins[value_area_levels.min()]
    value_area_high = bins[value_area_levels.max() + 1]
    
    # Current price position relative to Value Area
    current_price = data[close_col].iloc[-1]
    in_value_area = (current_price >= value_area_low) and (current_price <= value_area_high)
    above_va = current_price > value_area_high
    below_va = current_price < value_area_low
    
    return {
        'Volume_Profile': volume_profile,
        'Point_of_Control': poc_price,
        'Value_Area_Low': value_area_low,
        'Value_Area_High': value_area_high,
        'Current_Position': {
            'Price': current_price,
            'In_Value_Area': in_value_area,
            'Above_VA': above_va,
            'Below_VA': below_va,
            'Distance_from_POC': ((current_price - poc_price) / poc_price) * 100
        }
    }
```

**Explanation:**

The `calculate_volume_profile` function implements Volume Profile Analysis, a sophisticated technique that reveals where the majority of trading activity (volume) occurred at specific price levels, rather than just when.

**Conceptual Framework:**
Unlike time-based indicators that show what happened when, Volume Profile shows what happened where. It divides the price range into bins (levels) and sums the volume that occurred at each level. This creates a histogram showing "fair value" areas where buyers and sellers agreed most frequently.

**Key Components:**
1. **Point of Control (POC)**: The price level with the highest traded volume. In NEPSE, the POC acts as a "magnet" or gravitational center for price. When price moves away from POC but lacks volume support, it tends to revert back to this level. Institutional traders in Nepal often use POC as a reference for "fair value" entries.

2. **Value Area (VA)**: The price range containing approximately 70% of total volume (1 standard deviation in volume distribution). The Value Area High (VAH) and Value Area Low (VAL) act as dynamic support and resistance levels. For NEPSE:
   - Breakout above VAH with volume = Strong bullish, target next resistance
   - Breakdown below VAL with volume = Strong bearish, target next support
   - Price inside VA = Balanced, fair value, range-bound strategies work best

3. **Volume Profile Shape**: The distribution shape reveals market structure:
   - **D-shaped (bell curve)**: Balanced market, good for mean reversion
   - **P-shaped (high volume at bottom)**: Accumulation, bullish
   - **b-shaped (high volume at top)**: Distribution, bearish
   - **Thin profile**: Low liquidity, avoid large positions

**NEPSE Application:**
For Nepali stocks, Volume Profile is particularly valuable because:
- **Low Float Awareness**: Many NEPSE stocks have low free float. Volume Profile shows where major holders accumulated (thick volume areas) versus thin air (gaps) where price can move fast.
- **Institutional Footprints**: Large volume nodes often represent institutional accumulation zones. When price returns to these levels, institutions often defend their positions, creating support.
- **Breakout Validation**: In NEPSE, false breakouts are common. A breakout is only valid if price moves out of the Value Area on volume > 2x average, indicating genuine conviction rather than manipulation.

### **13.5.3 On-Balance Volume (OBV)**

OBV was covered in section 13.5.1. It serves as the primary volume trend indicator, cumulatively tracking whether volume is flowing into or out of a NEPSE stock.

---

## **13.6 Position and Rank Indicators**

Position and rank indicators normalize price data to statistical distributions, enabling comparison across different stocks and time periods regardless of absolute price levels.

### **13.6.1 Percentile Ranks**

Percentile ranks indicate where the current price stands relative to its historical range.

```python
def calculate_percentile_ranks(data, windows=[20, 50, 200], price_col='Close'):
    """
    Calculate percentile ranks for NEPSE stocks.
    
    Percentile Rank = (Number of values below current) / Total values * 100
    
    Interpretation:
    - 0-20: Bottom quintile (oversold, potential support)
    - 20-40: Lower quartile (weak)
    - 40-60: Middle (fair value)
    - 60-80: Upper quartile (strong)
    - 80-100: Top quintile (overbought, potential resistance)
    
    NEPSE Use:
    - Compare different stocks regardless of price level
    - Identify extreme readings for mean reversion
    - Sector rotation analysis (which sectors in top/bottom percentiles)
    """
    df = data.copy()
    
    for window in windows:
        # Calculate rolling percentile rank
        # Using scipy's percentileofscore for exact calculation
        from scipy.stats import percentileofscore
        
        def rolling_percentile(x):
            if len(x) < window:
                return np.nan
            return percentileofscore(x[:-1], x[-1], kind='rank')
        
        df[f'Percentile_{window}'] = df[price_col].rolling(window=window).apply(
            rolling_percentile, raw=True
        )
        
        # Alternative: Simple min-max scaling (0-100) within window
        rolling_min = df[price_col].rolling(window=window).min()
        rolling_max = df[price_col].rolling(window=window).max()
        df[f'Position_Score_{window}'] = ((df[price_col] - rolling_min) / 
                                           (rolling_max - rolling_min)) * 100
    
    # Composite percentile (average of different timeframes)
    df['Composite_Percentile'] = df[[f'Percentile_{w}' for w in windows]].mean(axis=1)
    
    # Percentile-based signals
    df['Extreme_Low'] = df['Composite_Percentile'] < 10   # Bottom decile
    df['Extreme_High'] = df['Composite_Percentile'] > 90  # Top decile
    df['Fair_Value'] = (df['Composite_Percentile'] > 40) & (df['Composite_Percentile'] < 60)
    
    return df

def calculate_percentile_divergence(data, price_col='Close', vol_col='Vol'):
    """
    Detect divergences between price percentiles and volume percentiles.
    
    Bullish: Price in low percentile but volume in high percentile (accumulation)
    Bearish: Price in high percentile but volume in high percentile (distribution)
    """
    # Calculate rolling percentiles for price and volume
    price_pct = data[price_col].rolling(20).apply(
        lambda x: percentileofscore(x[:-1], x[-1]), raw=True
    )
    vol_pct = data[vol_col].rolling(20).apply(
        lambda x: percentileofscore(x[:-1], x[-1]), raw=True
    )
    
    # Divergence detection
    bullish_div = (price_pct < 30) & (vol_pct > 70)  # Low price, high volume
    bearish_div = (price_pct > 70) & (vol_pct > 70)  # High price, high volume
    
    return pd.DataFrame({
        'Price_Percentile': price_pct,
        'Volume_Percentile': vol_pct,
        'Bullish_Divergence': bullish_div,
        'Bearish_Divergence': bearish_div
    })
```

**Explanation:**

The `calculate_percentile_ranks` function transforms absolute price levels into relative rankings within historical windows, solving a critical problem in NEPSE analysis: comparing stocks with different price levels (e.g., a Rs. 10,000 stock vs a Rs. 100 stock).

**Percentile Calculation:**
Using `percentileofscore` from scipy, the function calculates what percentage of historical prices (over the lookback window) are below the current price. A 90th percentile reading means the stock is trading higher than 90% of the past 20 (or 50, or 200) days—statistically overbought.

**NEPSE Multi-Timeframe Analysis:**
The function calculates percentiles across three timeframes:
- **20-day**: Short-term overbought/oversold (trading)
- **50-day**: Medium-term positioning (swing trading)
- **200-day**: Long-term secular trends (investing)

The Composite Percentile averages these, providing a consensus view of where the stock stands across all time horizons.

**Divergence Analysis:**
The `calculate_percentile_divergence` function identifies smart money activity by comparing price percentiles to volume percentiles. In NEPSE:
- **Bullish Divergence**: Price in bottom 30% (appears weak) but volume in top 30% (heavy accumulation). This suggests institutions are quietly accumulating shares without driving prices up—often precedes major rallies.
- **Bearish Divergence**: Price in top 30% (appears strong) with volume in top 30% (heavy distribution). Indicates institutions selling to retail investors at peaks—often marks tops.

### **13.6.2 Position in Range**

Position in Range (also known as Stochastic Oscillator) measures where the current price sits relative to the recent high-low range.

```python
def calculate_stochastic_indicators(data, k_window=14, d_window=3, 
                                   high_col='High', low_col='Low', close_col='Close'):
    """
    Calculate Stochastic Oscillator (%K and %D) for NEPSE momentum.
    
    %K = (Current Close - Lowest Low) / (Highest High - Lowest Low) * 100
    %D = 3-day SMA of %K (signal line)
    
    Interpretation:
    - >80: Overbought (price near top of range)
    - <20: Oversold (price near bottom of range)
    - %K crossing above %D: Buy signal
    - %K crossing below %D: Sell signal
    
    NEPSE Specific:
    - Use 14-period for weekly analysis (2 weeks of NEPSE trading)
    - Divergences highly predictive due to lower algorithmic trading
    """
    # Calculate %K (Fast Stochastic)
    lowest_low = data[low_col].rolling(window=k_window).min()
    highest_high = data[high_col].rolling(window=k_window).max()
    
    # Handle case where high == low (division by zero)
    range_hl = highest_high - lowest_low
    range_hl = range_hl.replace(0, np.nan)  # Avoid division by zero
    
    pct_k = ((data[close_col] - lowest_low) / range_hl) * 100
    
    # Calculate %D (Slow Stochastic) - signal line
    pct_d = pct_k.rolling(window=d_window).mean()
    
    # Slow Stochastic (smoothed %K)
    pct_k_slow = pct_k.rolling(window=3).mean()
    pct_d_slow = pct_k_slow.rolling(window=d_window).mean()
    
    # Stochastic RSI (Stochastics applied to RSI)
    rsi = calculate_rsi(data, window=14)  # From previous section
    rsi_low = rsi.rolling(window=k_window).min()
    rsi_high = rsi.rolling(window=k_window).max()
    stoch_rsi = ((rsi - rsi_low) / (rsi_high - rsi_low)) * 100
    
    # Signal generation
    overbought = pct_k > 80
    oversold = pct_k < 20
    
    # Crossovers
    k_cross_above_d = (pct_k > pct_d) & (pct_k.shift(1) <= pct_d.shift(1))
    k_cross_below_d = (pct_k < pct_d) & (pct_k.shift(1) >= pct_d.shift(1))
    
    # Divergences (price vs stochastic)
    price_high = data[close_col].rolling(window=5).max() == data[close_col]
    price_low = data[close_col].rolling(window=5).min() == data[close_col]
    
    stoch_high = pct_k.rolling(window=5).max() == pct_k
    stoch_low = pct_k.rolling(window=5).min() == pct_k
    
    # Bearish divergence: Price higher high, Stoch lower high
    bearish_div = price_high & (pct_k < pct_k.shift(5))
    
    # Bullish divergence: Price lower low, Stoch higher low
    bullish_div = price_low & (pct_k > pct_k.shift(5))
    
    return pd.DataFrame({
        'Stoch_K': pct_k,
        'Stoch_D': pct_d,
        'Stoch_K_Slow': pct_k_slow,
        'Stoch_D_Slow': pct_d_slow,
        'Stoch_RSI': stoch_rsi,
        'Stoch_Overbought': overbought,
        'Stoch_Oversold': oversold,
        'Stoch_Buy_Signal': k_cross_above_d & oversold,
        'Stoch_Sell_Signal': k_cross_below_d & overbought,
        'Stoch_Bullish_Div': bullish_div,
        'Stoch_Bearish_Div': bearish_div
    })
```

**Explanation:**

The `calculate_stochastic_indicators` function implements the Stochastic Oscillator family, including Fast Stochastic (%K), Slow Stochastic (%D), and Stochastic RSI. These indicators are particularly effective for NEPSE because they normalize price action to a 0-100 scale regardless of the stock's absolute price, enabling comparison across the diverse price ranges in the Nepali market (from Rs. 10 to Rs. 10,000+).

**%K Calculation:**
The raw Stochastic (%K) measures where the current close sits within the recent high-low range:
$$\%K = \frac{Close - Lowest Low}{Highest High - Lowest Low} \times 100$$

A reading of 90 means the stock closed in the top 10% of the 14-day range (overbought). A reading of 10 means it closed in the bottom 10% (oversold).

**Signal Generation:**
The function generates buy signals when %K crosses above %D (signal line) while in oversold territory (<20), indicating the first sign of buying pressure after a decline. Sell signals occur when %K crosses below %D in overbought territory (>80).

**Divergence Detection:**
For NEPSE specifically, the function detects divergences by comparing price peaks/troughs to Stochastic peaks/troughs over 5-day windows. Bearish divergence (price higher high, Stochastic lower high) is a reliable reversal warning in Nepali stocks because it indicates weakening buying pressure despite higher prices—often seen when retail investors chase highs while institutions distribute.

---

## **13.6 Position and Rank Indicators**

### **13.6.3 Z-Scores**

Z-Scores standardize data points to indicate how many standard deviations they are from the mean, enabling statistical arbitrage and mean reversion strategies.

```python
def calculate_zscore_indicators(data, windows=[20, 50, 200], price_col='Close'):
    """
    Calculate Z-Scores for statistical mean reversion analysis in NEPSE.
    
    Z-Score = (Current Price - Mean) / Standard Deviation
    
    Interpretation:
    - |Z| < 1: Within normal range (68% of data)
    - 1 < |Z| < 2: Moderate deviation (27% of data)
    - |Z| > 2: Extreme deviation (5% of data) - mean reversion candidate
    
    NEPSE Strategy:
    - Z > 2: Sell (price statistically extended, likely to revert)
    - Z < -2: Buy (price statistically depressed, likely to bounce)
    - Z crossing 0: Trend confirmation (momentum shift)
    """
    df = data.copy()
    results = pd.DataFrame(index=df.index)
    
    for window in windows:
        # Calculate rolling mean and std
        rolling_mean = df[price_col].rolling(window=window).mean()
        rolling_std = df[price_col].rolling(window=window).std()
        
        # Calculate Z-Score
        zscore = (df[price_col] - rolling_mean) / rolling_std
        
        results[f'ZScore_{window}'] = zscore
        
        # Statistical thresholds
        results[f'ZScore_Extreme_High_{window}'] = zscore > 2
        results[f'ZScore_Extreme_Low_{window}'] = zscore < -2
        results[f'ZScore_Normal_{window}'] = abs(zscore) < 1
        
        # Mean reversion signals
        results[f'ZScore_MeanRev_Buy_{window}'] = (zscore < -2) & (zscore.diff() > 0)
        results[f'ZScore_MeanRev_Sell_{window}'] = (zscore > 2) & (zscore.diff() < 0)
    
    # Composite Z-Score (average of multiple timeframes)
    zscore_cols = [f'ZScore_{w}' for w in windows]
    results['ZScore_Composite'] = results[zscore_cols].mean(axis=1)
    
    # Z-Score momentum (rate of change of Z-Score)
    results['ZScore_Momentum'] = results['ZScore_Composite'].diff(5)
    
    # Z-Score trend (is it reverting or extending?)
    results['ZScore_Trend'] = np.where(
        results['ZScore_Composite'] > 1.5, 'Overbought',
        np.where(results['ZScore_Composite'] < -1.5, 'Oversold', 'Neutral')
    )
    
    return results

def calculate_zscore_divergence(data, price_col='Close', window=20):
    """
    Detect divergences between price and its Z-Score for NEPSE.
    
    Normal divergence: Price makes new high, Z-Score makes lower high
    (Price extending but statistical momentum weakening)
    """
    zscore = calculate_zscore_indicators(data, windows=[window])
    zscore_col = f'ZScore_{window}'
    
    # Find peaks and troughs
    price_high = data[price_col] == data[price_col].rolling(window=5, center=True).max()
    price_low = data[price_col] == data[price_col].rolling(window=5, center=True).min()
    
    zscore_high = zscore[zscore_col] == zscore[zscore_col].rolling(window=5, center=True).max()
    zscore_low = zscore[zscore_col] == zscore[zscore_col].rolling(window=5, center=True).min()
    
    # Divergences
    bearish_div = price_high & (zscore[zscore_col] < zscore[zscore_col].shift(5))
    bullish_div = price_low & (zscore[zscore_col] > zscore[zscore_col].shift(5))
    
    return pd.DataFrame({
        'ZScore': zscore[zscore_col],
        'Bearish_Divergence': bearish_div,
        'Bullish_Divergence': bullish_div,
        'Is_Peak': price_high,
        'Is_Trough': price_low
    })
```

**Explanation:**

The `calculate_zscore_indicators` function standardizes price data to a normal distribution with mean 0 and standard deviation 1, enabling statistical arbitrage strategies in NEPSE.

**Statistical Foundation:**
The Z-Score formula $Z = \frac{X - \mu}{\sigma}$ transforms raw prices into standard deviations from the mean. In a normal distribution:
- 68% of data falls within $|Z| < 1$
- 95% within $|Z| < 2$
- 99.7% within $|Z| < 3$

**NEPSE Mean Reversion Strategy:**
For Nepali stocks, which often exhibit mean-reverting behavior due to limited float and retail participation, extreme Z-Scores present high-probability trading opportunities:
- **Z < -2**: Price is 2+ standard deviations below the mean. Statistically, there's a 95% probability it will revert toward the mean within the lookback window. This is a high-probability buy signal for NEPSE mean reversion strategies.
- **Z > +2**: Price is 2+ standard deviations above the mean. 95% probability of reversion. Sell signal or short opportunity (if available).

**Multi-Timeframe Composite:**
The function calculates Z-Scores across 20, 50, and 200-day windows, then averages them into a Composite Z-Score. This prevents false signals from single timeframes. For example, a stock might be overbought short-term (20-day Z > 2) but oversold long-term (200-day Z < -1). The composite provides a balanced view.

**Z-Score Momentum:**
The rate of change of the Z-Score (`ZScore_Momentum`) indicates whether the stock is accelerating toward extremes (momentum increasing) or reverting toward mean (momentum decreasing). Positive momentum while Z > 1.5 indicates strong trend continuation; negative momentum while Z > 1.5 indicates trend exhaustion.

---

## **13.7 Cross-Domain Application**

While this chapter focuses on NEPSE financial data, the indicator engineering principles apply across domains:

**Weather Forecasting:**
- **Trend Indicators**: Moving averages of temperature to detect climate change trends
- **Momentum**: Rate of change in atmospheric pressure to predict storm intensity
- **Volatility**: Standard deviation of temperature (weather instability)

**Healthcare (Patient Monitoring):**
- **Trend**: Moving average of heart rate to detect gradual deterioration
- **Momentum**: Rate of change in blood pressure for acute event detection
- **Z-Scores**: Standardized vital signs to detect anomalies across different patient baselines

**IoT/Sensor Data:**
- **Trend**: SMA of vibration sensors to detect bearing wear in machinery
- **Volatility**: ATR equivalent for sensor range to detect instability
- **Volume**: Event frequency (counts) as intensity indicator

The mathematical foundations (rolling windows, standardization, momentum calculations) remain consistent; only the interpretation changes based on domain context.

---

## **13.8 Indicator Computation Libraries**

For production NEPSE prediction systems, use optimized libraries rather than custom implementations:

```python
# Recommended libraries for NEPSE indicator calculation

# 1. TA-Lib (Technical Analysis Library)
# Fast C implementation, industry standard
import talib

def calculate_with_talib(df):
    """Using TA-Lib for production NEPSE systems"""
    # Moving Averages
    sma20 = talib.SMA(df['Close'], timeperiod=20)
    ema12 = talib.EMA(df['Close'], timeperiod=12)
    ema26 = talib.EMA(df['Close'], timeperiod=26)
    
    # Momentum
    rsi = talib.RSI(df['Close'], timeperiod=14)
    macd, macd_signal, macd_hist = talib.MACD(
        df['Close'], fastperiod=12, slowperiod=26, signalperiod=9
    )
    
    # Volatility
    upper, middle, lower = talib.BBANDS(
        df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2
    )
    atr = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    # Volume
    obv = talib.OBV(df['Close'], df['Vol'])
    
    return pd.DataFrame({
        'SMA_20': sma20, 'EMA_12': ema12, 'EMA_26': ema26,
        'RSI': rsi, 'MACD': macd, 'MACD_Signal': macd_signal,
        'BB_Upper': upper, 'BB_Lower': lower, 'ATR': atr, 'OBV': obv
    })

# 2. pandas-ta (Pure Python, extensive indicator library)
import pandas_ta as ta

def calculate_with_pandas_ta(df):
    """Using pandas-ta for flexible NEPSE analysis"""
    # Add all common indicators at once
    df.ta.rsi(length=14, append=True)
    df.ta.macd(fast=12, slow=26, signal=9, append=True)
    df.ta.bbands(length=20, std=2, append=True)
    df.ta.atr(length=14, append=True)
    df.ta.obv(append=True)
    df.ta.adx(length=14, append=True)
    df.ta.stoch(k=14, d=3, append=True)
    
    # Custom strategy
    my_strategy = ta.Strategy(
        name="NEPSE Momentum",
        description="SMA, RSI, MACD, and BBands",
        ta=[
            {"kind": "sma", "length": 20},
            {"kind": "sma", "length": 50},
            {"kind": "rsi", "length": 14},
            {"kind": "macd", "fast": 12, "slow": 26},
            {"kind": "bbands", "length": 20},
        ],
    )
    
    df.ta.strategy(my_strategy)
    return df

# 3. Custom optimized functions for production
# Use Numba for JIT compilation if performance critical
from numba import jit

@jit(nopython=True)
def fast_sma(prices, window):
    """Numba-accelerated SMA for high-frequency NEPSE data"""
    n = len(prices)
    result = np.empty(n)
    result[:window-1] = np.nan
    
    # First value
    result[window-1] = np.mean(prices[:window])
    
    # Efficient rolling calculation
    for i in range(window, n):
        result[i] = result[i-1] + (prices[i] - prices[i-window]) / window
    
    return result
```

**Explanation:**

This section provides production-ready implementations using industry-standard libraries. For NEPSE prediction systems handling large datasets or real-time streaming, performance optimization is crucial.

**TA-Lib (Technical Analysis Library):**
TA-Lib is the industry standard written in C, offering 200+ indicators with optimized performance. For NEPSE production systems, TA-Lib is preferred because:
- **Speed**: C implementation is 100x faster than pure Python
- **Accuracy**: Uses established financial formulas (Wilder's smoothing, etc.)
- **Memory Efficiency**: Processes data in C arrays, not Python objects

The example shows how to calculate core NEPSE indicators: SMA, EMA, RSI, MACD, Bollinger Bands, ATR, and OBV in a single efficient pass.

**pandas-ta:**
A pure Python alternative offering 130+ indicators with a pandas-native interface. Advantages for NEPSE analysis:
- **Flexibility**: Easy to customize parameters for Nepali market conditions
- **Strategy Objects**: Can bundle multiple indicators into reusable strategies
- **Pandas Integration**: Returns DataFrames that integrate seamlessly with existing NEPSE data pipelines

**Numba Optimization:**
For high-frequency NEPSE data (tick data or 1-minute bars), the `@jit` decorator compiles Python functions to machine code at runtime. The `fast_sma` example shows a rolling mean calculation optimized for speed—critical when processing thousands of NEPSE stocks in real-time.

---

## **13.9 Indicator Selection Framework**

With hundreds of indicators available, selecting the right ones for NEPSE prediction requires a systematic framework.

```python
class NEPSEIndicatorSelector:
    """
    Framework for selecting optimal indicators for NEPSE prediction models.
    
    Selection Criteria:
    1. Predictive Power: Correlation with future returns
    2. Non-Redundancy: Low correlation with other selected indicators
    3. Robustness: Performance across different market regimes
    4. Computational Efficiency: Calculation speed for real-time use
    """
    
    def __init__(self, data):
        self.data = data
        self.indicators = {}
        self.performance_metrics = {}
        
    def generate_candidate_indicators(self):
        """Generate comprehensive set of indicators for NEPSE"""
        # Trend indicators
        self.indicators['SMA_20'] = self.data['Close'].rolling(20).mean()
        self.indicators['EMA_12'] = self.data['Close'].ewm(span=12).mean()
        self.indicators['ADX'] = calculate_adx(self.data)['ADX']
        
        # Momentum
        self.indicators['RSI'] = calculate_rsi(self.data)
        self.indicators['MACD'] = calculate_macd(self.data)['MACD']
        self.indicators['ROC'] = calculate_roc(self.data)
        
        # Volatility
        self.indicators['ATR'] = calculate_atr(self.data)['ATR']
        self.indicators['BB_Width'] = calculate_bollinger_bands(self.data)['BB_Width']
        
        # Volume
        self.indicators['OBV'] = calculate_volume_indicators(self.data)['OBV']
        self.indicators['Volume_Ratio'] = calculate_volume_indicators(self.data)['Volume_Ratio']
        
        # Position
        self.indicators['Percentile_20'] = calculate_percentile_ranks(self.data, windows=[20])['Percentile_20']
        self.indicators['Stoch_K'] = calculate_stochastic_indicators(self.data)['Stoch_K']
        
        return pd.DataFrame(self.indicators)
    
    def evaluate_predictive_power(self, indicator_df, forward_periods=[1, 5, 10]):
        """
        Evaluate how well each indicator predicts future returns.
        
        Uses Spearman correlation (rank correlation) to handle non-linear relationships.
        """
        future_returns = {}
        
        for period in forward_periods:
            future_returns[f'Return_{period}d'] = self.data['Close'].pct_change(period).shift(-period)
        
        returns_df = pd.DataFrame(future_returns)
        
        correlations = {}
        for col in indicator_df.columns:
            if indicator_df[col].isna().all():
                continue
                
            correlations[col] = {}
            for ret_col in returns_df.columns:
                # Spearman correlation (monotonic relationships)
                corr = indicator_df[col].corr(returns_df[ret_col], method='spearman')
                correlations[col][ret_col] = corr
        
        return pd.DataFrame(correlations).T
    
    def select_non_redundant_indicators(self, indicator_df, correlation_threshold=0.85):
        """
        Remove highly correlated indicators to avoid multicollinearity.
        
        Uses hierarchical clustering to group similar indicators and select
        the best representative from each group.
        """
        from scipy.cluster import hierarchy
        from scipy.spatial.distance import squareform
        
        # Calculate correlation matrix
        corr_matrix = indicator_df.corr().abs()
        
        # Convert to distance matrix
        distance_matrix = 1 - corr_matrix
        
        # Hierarchical clustering
        linkage = hierarchy.linkage(squareform(distance_matrix), method='average')
        
        # Form flat clusters
        clusters = hierarchy.fcluster(linkage, t=correlation_threshold, criterion='distance')
        
        # Select one indicator per cluster (highest predictive power)
        selected_indicators = []
        for cluster_id in np.unique(clusters):
            cluster_indicators = indicator_df.columns[clusters == cluster_id]
            # Select the one with highest variance (most information)
            variances = indicator_df[cluster_indicators].var()
            selected = variances.idxmax()
            selected_indicators.append(selected)
        
        return selected_indicators, clusters
    
    def get_optimal_indicator_set(self):
        """Execute full selection pipeline"""
        # Generate all candidates
        indicator_df = self.generate_candidate_indicators()
        
        # Evaluate predictive power
        predictive_power = self.evaluate_predictive_power(indicator_df)
        
        # Remove redundant indicators
        selected, clusters = self.select_non_redundant_indicators(indicator_df)
        
        return {
            'selected_indicators': selected,
            'predictive_power': predictive_power,
            'clusters': clusters,
            'indicator_data': indicator_df[selected]
        }

# Usage for NEPSE:
# selector = NEPSEIndicatorSelector(nepse_df)
# optimal_indicators = selector.get_optimal_indicator_set()
```

**Explanation:**

The `NEPSEIndicatorSelector` class provides a systematic framework for selecting the optimal subset of technical indicators for machine learning models, addressing the curse of dimensionality and multicollinearity common in financial prediction.

**Phase 1: Candidate Generation**
The framework generates 20+ standard indicators covering all categories (trend, momentum, volatility, volume, position). This ensures comprehensive coverage of potential predictive signals in NEPSE data.

**Phase 2: Predictive Power Evaluation**
Using Spearman correlation (which captures monotonic but non-linear relationships), the framework tests each indicator's correlation with forward returns (1, 5, and 10 days ahead). Spearman is preferred over Pearson because financial relationships are rarely linear but often rank-correlated. For example, extremely high RSI (>80) may consistently predict negative returns, but the relationship isn't linear across all RSI values.

**Phase 3: Non-Redundancy Selection**
Financial indicators are highly correlated (e.g., RSI and Stochastic often move together). Including all of them in a model creates multicollinearity, making coefficients unstable and reducing generalization. The framework uses hierarchical clustering on the correlation matrix to group similar indicators, then selects the highest-variance (most informative) member from each cluster. This ensures the final indicator set captures diverse aspects of market behavior without redundancy.

**NEPSE Optimization:**
For Nepali stocks, this selection process typically retains:
- **Trend**: ADX (trend strength) rather than multiple MAs
- **Momentum**: RSI (most robust) rather than Stochastic (noisy in low liquidity)
- **Volatility**: ATR (used for position sizing) rather than Bollinger Width
- **Volume**: Volume Ratio (simple but effective) rather than complex OBV derivatives

This optimized set of 4-6 indicators provides maximum predictive power with minimum computational overhead and collinearity for NEPSE prediction models.

---

## **13.7 Cross-Domain Application**

While this chapter focuses on NEPSE financial data, the indicator engineering principles apply universally:

**Retail Demand Forecasting:**
- **Trend Indicators**: 7-day and 30-day moving averages of sales to detect seasonal trends
- **Momentum**: Rate of change in foot traffic to predict inventory needs
- **Volatility**: Standard deviation of daily sales to assess business risk
- **Volume**: Transaction counts confirming price trends (average ticket size)

**Energy Consumption:**
- **Trend**: Moving averages of kWh usage to detect efficiency improvements
- **Momentum**: Acceleration in consumption indicating equipment malfunction
- **Z-Scores**: Statistical detection of anomalous consumption spikes

**Manufacturing/IoT:**
- **Bollinger Bands**: Sensor readings to detect when machines drift out of spec
- **RSI**: Vibration sensor momentum to predict bearing failure
- **ATR**: Range of temperature fluctuations indicating system instability

The mathematical transformations (smoothing, normalization, rate-of-change, standardization) are domain-agnostic and form the foundation of time-series feature engineering across all industries.

---

## **13.8 Indicator Computation Libraries**

For production NEPSE systems, leverage these optimized libraries:

1. **TA-Lib**: C-based, 200+ indicators, fastest execution
2. **pandas-ta**: Pure Python, 130+ indicators, pandas integration
3. **ta**: Lightweight Python technical analysis library
4. **Numba**: JIT compilation for custom indicator acceleration
5. **Dask**: Parallel computation for multi-stock NEPSE screening

---

## **13.9 Indicator Selection Framework**

(See detailed implementation in section 13.6.3 with the `NEPSEIndicatorSelector` class)

Key principles:
1. **Predictive Power**: Correlation with forward returns
2. **Orthogonality**: Low inter-correlation (diversification of signals)
3. **Robustness**: Consistent performance across bull/bear markets
4. **Latency**: Computational efficiency for real-time NEPSE trading

---

## **13.10 Custom Indicator Development**

When standard indicators fail to capture NEPSE-specific patterns, develop custom indicators:

**NEPSE-Specific Custom Indicator Example:**
```python
def calculate_nepse_liquidity_score(data, vol_col='Vol', close_col='Close'):
    """
    Custom indicator for NEPSE liquidity assessment.
    
    Combines:
    - Volume relative to 20-day average
    - Price impact (how much price moves per unit volume)
    - Bid-ask spread proxy (High-Low relative to Close)
    
    Returns 0-100 score where:
    - 0-30: Illiquid (hard to enter/exit without moving price)
    - 30-70: Moderate liquidity
    - 70-100: High liquidity (institutional grade)
    """
    # Volume score (0-40 points)
    vol_ratio = data[vol_col] / data[vol_col].rolling(20).mean()
    vol_score = np.clip(vol_ratio * 20, 0, 40)  # Cap at 40
    
    # Price efficiency (0-30 points)
    # Lower price impact per volume = higher liquidity
    price_change = abs(data[close_col].pct_change())
    volume_normalized = data[vol_col] / data[vol_col].rolling(50).mean()
    price_impact = price_change / (volume_normalized + 0.001)
    efficiency_score = np.clip(30 - (price_impact * 100), 0, 30)
    
    # Range stability (0-30 points)
    # Consistent daily ranges indicate liquid, orderly markets
    daily_range = (data['High'] - data['Low']) / data['Close']
    range_consistency = 1 / (daily_range.rolling(10).std() + 0.01)
    stability_score = np.clip(range_consistency * 3, 0, 30)
    
    # Composite score
    liquidity_score = vol_score + efficiency_score + stability_score
    
    return pd.DataFrame({
        'Liquidity_Score': liquidity_score,
        'Volume_Component': vol_score,
        'Efficiency_Component': efficiency_score,
        'Stability_Component': stability_score,
        'Liquidity_Grade': pd.cut(liquidity_score, 
                                bins=[0, 30, 70, 100],
                                labels=['Low', 'Medium', 'High'])
    })
```

**Explanation:**

The `calculate_nepse_liquidity_score` function demonstrates custom indicator development tailored to the unique characteristics of the Nepal Stock Exchange. Standard liquidity indicators (like Amihud or bid-ask spreads) require tick-level data unavailable in standard NEPSE CSV feeds. This custom indicator reconstructs liquidity proxies from OHLCV data.

**Component Breakdown:**

1. **Volume Component (40 points)**: Measures relative volume compared to 20-day average. In NEPSE, liquidity is highly variable—some days see 10x normal volume during news events, other days see 0.1x during holidays or political uncertainty. This component scores high when volume is robust, indicating institutional participation and ease of execution.

2. **Price Efficiency Component (30 points)**: Measures price impact per unit of volume. In liquid markets, large volume should move price minimally. The formula $\frac{\Delta P}{Volume}$ measures this impact. Low impact = high efficiency = high liquidity score. This identifies NEPSE stocks where you can enter/exit large positions without significant slippage.

3. **Range Stability Component (30 points)**: Measures consistency of daily trading ranges. Liquid stocks exhibit consistent, orderly volatility. Illiquid NEPSE stocks often show erratic ranges—some days 1%, other days 10% on minimal volume. Low standard deviation of ranges indicates professional market making and liquidity.

**Composite Scoring:**
The final 0-100 score categorizes NEPSE stocks into:
- **Low (0-30)**: Avoid for large positions. High slippage, erratic fills, wide spreads. Suitable only for long-term buy-and-hold.
- **Medium (30-70)**: Acceptable for normal trading. Moderate slippage on entries/exits.
- **High (70-100)**: Institutional grade. Tight spreads, deep order books, minimal market impact. Suitable for large positions and algorithmic execution.

This custom indicator enables quantitative screening of the entire NEPSE universe to identify which stocks are tradable given specific position size constraints and execution requirements.

---

**End of Chapter 13**

This chapter covered the engineering of technical indicators specifically tailored for the NEPSE time-series prediction system, including trend, momentum, volatility, volume, and position indicators with detailed mathematical foundations and Python implementations.

